#### [Raw data](http://www.soest.hawaii.edu/pibhmc/cms/data-by-location/american-samoa/)

- *Observational*: all in percent cover; based on my exploratory analyses, I may be able to model abundance vs. exclusively presence/absence.
    1. SCLER_CORA: includes all corals. The others are just growth forms.
    2. MASSIVE
    3. PLATE_LIKE
    4. ENCRUSTING
    5. BRANCHING
    6. COLUMNAR
    7. FREE_LIVIN

Note that [Rose](http://www.soest.hawaii.edu/pibhmc/cms/data-by-location/american-samoa/rose-atoll/rose-atoll-habitat/) does not have optical validation data, but does have biological characterizations from prior surveys. This may be useful for predictive comparison only.

Note that Swains does not have any biological layers available.

- *Environmental*: 1-6 from PIBHMC site
    1. Depth (from bty: "*_bty")
    2. Hard vs. soft bottom (from Arc ASCII file: "*_hs")
    3. Rugosity (from Arc ASCII file: "*_rugosity")
    4. Slope (from Arc ASCII file: "*_slope")
    5. PASS 3/20: this attribute is rarely identified as influential...I'm going to exclude it for now..Curvature (from bty)
    6. Aspect (from bty)
    7. PASS 3/19: Buildings (from PacIOOS: "*_bldngs*"; not available for Rose, Swains). Not loading properly...ignore for now
    8. Environmental sensitivity index (from PacIOOS: "AS_all/Env_sens_index")
    9. Proximity to villages > 3rd quantile (acres) (from PacIOOS: "AS_all/Villages/")
    10. Distance from shore (from PacIOOS: "AS_all/Shorelines/")
    11. Wetland proximity (from PacIOOS: "AS_all/Wetlands/")
    12. PASS 4/15: Population 1960-2000 (from PacIOOS: "AS_all/Population/") *note*: this is actually only Tutuila and therefore is not very useful (i.e., black corals are verrrry far from the high population points). Instead looking at villages > 3rd quantile in acres
    13. ROMS u velocity (to E) (3 km resolution; from PacIOOS: "AS_all/Currents")
    14. ROMS v velocity (to N) (3 km resolution; from PacIOOS: "AS_all/Currents")

Priority locations:  Ofu-Olosega, Ta‘u, Tutuila
Secondary locations: Rose Atoll, Swains Island

All PIBHMC data is available at 5 m resolution for all locations except for Swains Island (5 m for bty, 40 m for hs, rugosity, slope). Using integrated data when available for completeness of coverage.

#### Load libraries

In [2]:
library(data.table)
library(geojsonio)
library(GISTools)
library(ncdf4)
library(raster)
library(rgdal)
library(rgeos)
library(sf)
library(spatial.tools)
library(tidyverse)

#### Load raw data

I need to...
- [done] load the observational and predictor data for each region (OfuOlo, Rose, Swains, Tau, Tut) and all regions (AS_all)
- [done] choose prj, ensure projections align
- [done] exclude depths < 30 m and > 200 m**
- [done] crop rasters
- [done] resample rasters to 50 m resolution
- [done] load netcdfs, reproject, save as points shp

## Ofu-Olo

In [3]:
setwd("C:/Users/linds/OneDrive/Documents/samoa_corals_data/")

# Ofu-Olosega
oo_bty <-raster('OfuOlo/OfuOlo_bty/oo_dbmb_mos4.asc')
oo_hs <- raster('OfuOlo/OfuOlo_hs/ofu_5m_hs_int.asc')
oo_rug <- raster('OfuOlo/OfuOlo_rugosity/OfuOlo_5m_Rugosity.asc')
oo_slope <- raster('OfuOlo/OfuOlo_slope/OfuOlo_5m_Slope.asc')
oo_TOAD <- st_read('OfuOlo/OfuOlo_TOAD/OFU_TOAD_ALL2.shp') # for reading the attribute table

# Assign projection
proj4string(oo_bty) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
proj4string(oo_hs) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
proj4string(oo_rug) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
proj4string(oo_slope) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
oo_TOAD <- st_transform(oo_TOAD, crs= st_crs(oo_bty))

# Create aspect raster
oo_aspect <- terrain(oo_bty, opt = "aspect", unit = "degrees")

Reading layer `OFU_TOAD_ALL2' from data source `C:\Users\linds\OneDrive\Documents\samoa_corals_data\OfuOlo\OfuOlo_TOAD\OFU_TOAD_ALL2.shp' using driver `ESRI Shapefile'
Simple feature collection with 3042 features and 97 fields
geometry type:  POINT
dimension:      XY
bbox:           xmin: -169.6982 ymin: -14.19928 xmax: -169.5957 ymax: -14.13887
epsg (SRID):    4326
proj4string:    +proj=longlat +datum=WGS84 +no_defs


#### Clip bathymetry raster < 30 m, > 200 m; crop others to same extent; resample rasters

In [4]:
oo_bty[oo_bty > -30] <- NA
oo_bty[oo_bty < -200] <- NA

oo_hs <- crop(oo_hs, oo_bty)
oo_rug <- crop(oo_rug, oo_bty)
oo_slope <- crop(oo_slope, oo_bty)
oo_aspect <- crop(oo_aspect, oo_bty)

# Resample rasters to a resolution of 50 m (factor of 10)
oo_bty <- aggregate(oo_bty, fact=10)
oo_hs <- aggregate(oo_hs, fact=10)
oo_rug <- aggregate(oo_rug, fact=10)
oo_slope <- aggregate(oo_slope, fact=10)
oo_aspect <- aggregate(oo_aspect, fact=10)

# Write tranformed rasters and shp to file
writeRaster(oo_bty, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_bty',
                                       "oo_bty.grd"), format='raster', overwrite=TRUE)
writeRaster(oo_hs, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_hs',
                                       "oo_hs.grd"), format='raster', overwrite=TRUE)
writeRaster(oo_rug, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_rugosity',
                                       "oo_rug.grd"), format='raster', overwrite=TRUE)
writeRaster(oo_slope, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_slope',
                                       "oo_slope.grd"), format='raster', overwrite=TRUE)
writeRaster(oo_aspect, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_asp',
                                       "oo_aspect.grd"), format='raster', overwrite=TRUE)
st_write(obj=oo_TOAD, dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD', 
         update=TRUE, driver="ESRI Shapefile", 'oo_TOAD.shp')

class      : RasterLayer 
dimensions : 154, 262, 40348  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 639745, 652845, 8429350, 8437050  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_bty/oo_bty.grd 
names      : oo_dbmb_mos4 
values     : -199.935, -30.0152  (min, max)


class      : RasterLayer 
dimensions : 154, 262, 40348  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 639745, 652845, 8429350, 8437050  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_hs/oo_hs.grd 
names      : ofu_5m_hs_int 
values     : 1, 2  (min, max)


class      : RasterLayer 
dimensions : 153, 262, 40086  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 639750, 652850, 8429395, 8437045  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_rugosity/oo_rug.grd 
names      : OfuOlo_5m_Rugosity 
values     : 1.000045, 4.973554  (min, max)


class      : RasterLayer 
dimensions : 154, 262, 40348  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 639745, 652845, 8429350, 8437050  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_slope/oo_slope.grd 
names      : OfuOlo_5m_Slope 
values     : 0, 69.63569  (min, max)


class      : RasterLayer 
dimensions : 154, 262, 40348  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 639745, 652845, 8429350, 8437050  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_asp/oo_aspect.grd 
names      : aspect 
values     : 0.8065008, 359.3722  (min, max)


Updating layer `oo_TOAD' to data source `C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD' using driver `ESRI Shapefile'
Updating existing layer oo_TOAD
Writing 3042 features with 97 fields and geometry type Point.


## Tau

In [5]:
# Tau
tau_bty <-raster('Tau/Tau_bty/tau_dbmb_mos.asc')
tau_hs <- raster('Tau/Tau_hs/tau_5m_hs_int.asc')
tau_rug <- raster('Tau/Tau_rugosity/Tau_5m_Rugosity.asc')
tau_slope <- raster('Tau/Tau_slope/Tau_5m_Slope.asc')
tau_TOAD <- st_read('Tau/Tau_TOAD/Toad_Tau_all2.shp')
                    
proj4string(tau_bty) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
proj4string(tau_hs) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
proj4string(tau_rug) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
proj4string(tau_slope) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
tau_TOAD <- st_transform(tau_TOAD, crs= st_crs(tau_bty))

tau_aspect <- terrain(tau_bty, opt = "aspect", unit = "degrees")

# tau_bty # view raster attributes                   

Reading layer `Toad_Tau_all2' from data source `C:\Users\linds\OneDrive\Documents\samoa_corals_data\Tau\Tau_TOAD\Toad_Tau_all2.shp' using driver `ESRI Shapefile'
Simple feature collection with 1276 features and 97 fields
geometry type:  POINT
dimension:      XY
bbox:           xmin: -169.553 ymin: -14.28124 xmax: -169.4154 ymax: -14.20876
epsg (SRID):    4326
proj4string:    +proj=longlat +datum=WGS84 +no_defs


#### Clip bathymetry raster < 30 m, > 200 m; crop others to same extent; resample rasters

In [6]:
tau_bty[tau_bty > -30] <- NA
tau_bty[tau_bty < -200] <- NA

tau_hs <- crop(tau_hs, tau_bty)
tau_rug <- crop(tau_rug, tau_bty)
tau_slope <- crop(tau_slope, tau_bty)
tau_aspect <- crop(tau_aspect, tau_bty)

# Resample rasters to a resolution of 50 m (factor of 10)
tau_bty <- aggregate(tau_bty, fact=10)
tau_hs <- aggregate(tau_hs, fact=10)
tau_rug <- aggregate(tau_rug, fact=10)
tau_slope <- aggregate(tau_slope, fact=10)
tau_aspect <- aggregate(tau_aspect, fact=10)

# Write tranformed rasters and shp to file
writeRaster(tau_bty, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_bty',
                                       "tau_bty.grd"), format='raster', overwrite=TRUE)
writeRaster(tau_hs, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_hs',
                                       "tau_hs.grd"), format='raster', overwrite=TRUE)
writeRaster(tau_rug, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_rugosity',
                                       "tau_rug.grd"), format='raster', overwrite=TRUE)
writeRaster(tau_slope, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_slope',
                                       "tau_slope.grd"), format='raster', overwrite=TRUE)
writeRaster(tau_aspect, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_asp',
                                       "tau_aspect.grd"), format='raster', overwrite=TRUE)
st_write(obj=tau_TOAD, dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD', 
         update=TRUE, driver="ESRI Shapefile", 'tau_TOAD.shp')

class      : RasterLayer 
dimensions : 219, 401, 87819  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 651350, 671400, 8420175, 8431125  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_bty/tau_bty.grd 
names      : tau_dbmb_mos 
values     : -199.991, -30.0361  (min, max)


class      : RasterLayer 
dimensions : 219, 401, 87819  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 651352, 671402, 8420177, 8431127  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_hs/tau_hs.grd 
names      : tau_5m_hs_int 
values     : 1, 2  (min, max)


class      : RasterLayer 
dimensions : 175, 296, 51800  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 656545.7, 671345.7, 8420362, 8429112  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_rugosity/tau_rug.grd 
names      : Tau_5m_Rugosity 
values     : 1.000437, 5.440688  (min, max)


class      : RasterLayer 
dimensions : 219, 401, 87819  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 651350, 671400, 8420175, 8431125  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_slope/tau_slope.grd 
names      : Tau_5m_Slope 
values     : 0.1441366, 61.96326  (min, max)


class      : RasterLayer 
dimensions : 219, 401, 87819  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 651350, 671400, 8420175, 8431125  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_asp/tau_aspect.grd 
names      : aspect 
values     : 5.783459, 353.7966  (min, max)


Updating layer `tau_TOAD' to data source `C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD' using driver `ESRI Shapefile'
Updating existing layer tau_TOAD
Writing 1276 features with 97 fields and geometry type Point.


## Tutuila

In [7]:
tut_bty <-raster('Tut/Tut_bty/tut_dbmb.asc')
tut_hs <- raster('Tut/Tut_hs/tut-re8101-unspv-5m.tif')
tut_rug <- raster('Tut/Tut_rugosity/Tutuila_5m_Rugosity.asc')
tut_slope <- raster('Tut/Tut_slope/Tutuila_5m_Slope.asc')
tut_TOAD <- st_read('Tut/Tut_TOAD/tut_toad_all.shp')

proj4string(tut_bty) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
proj4string(tut_hs) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
proj4string(tut_rug) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
proj4string(tut_slope) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
tut_TOAD <- st_transform(tut_TOAD, crs= st_crs(tut_bty))

tut_aspect <- terrain(tut_bty, opt = "aspect", unit = "degrees")

Reading layer `tut_toad_all' from data source `C:\Users\linds\OneDrive\Documents\samoa_corals_data\Tut\Tut_TOAD\tut_toad_all.shp' using driver `ESRI Shapefile'
Simple feature collection with 5602 features and 107 fields
geometry type:  POINT
dimension:      XY
bbox:           xmin: -170.9227 ymin: -14.3715 xmax: -170.4948 ymax: -14.20533
epsg (SRID):    4267
proj4string:    +proj=longlat +ellps=clrk66 +no_defs


#### Clip bathymetry raster < 30 m, > 200 m; crop others to same extent; resample rasters

In [8]:
tut_bty[tut_bty > -30] <- NA
tut_bty[tut_bty < -200] <- NA

tut_hs <- crop(tut_hs, tut_bty)
tut_rug <- crop(tut_rug, tut_bty)
tut_slope <- crop(tut_slope, tut_bty)
tut_aspect <- crop(tut_aspect, tut_bty)

tut_bty <- aggregate(tut_bty, fact=10)
tut_hs <- aggregate(tut_hs, fact=10)
tut_rug <- aggregate(tut_rug, fact=10)
tut_slope <- aggregate(tut_slope, fact=10)
tut_aspect <- aggregate(tut_aspect, fact=10)

# Write tranformed rasters and shp to file
writeRaster(tut_bty, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_bty',
                                       "tut_bty.grd"), format='raster', overwrite=TRUE)
writeRaster(tut_hs, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_hs',
                                       "tut_hs.grd"), format='raster', overwrite=TRUE)
writeRaster(tut_rug, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_rugosity',
                                       "tut_rug.grd"), format='raster', overwrite=TRUE)
writeRaster(tut_slope, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_slope',
                                       "tut_slope.grd"), format='raster', overwrite=TRUE)
writeRaster(tut_aspect, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_asp',
                                       "tut_aspect.grd"), format='raster', overwrite=TRUE)
st_write(obj=tut_TOAD, dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD', 
         update=TRUE, driver="ESRI Shapefile", 'tut_TOAD.shp')

class      : RasterLayer 
dimensions : 417, 956, 398652  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 507747.5, 555547.5, 8409758, 8430608  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_bty/tut_bty.grd 
names      : tut_dbmb 
values     : -200, -30  (min, max)


class      : RasterLayer 
dimensions : 416, 905, 376480  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 510336.2, 555586.2, 8409803, 8430603  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_hs/tut_hs.grd 
names      : tut.re8101.unspv.5m 
values     : 0, 2  (min, max)


class      : RasterLayer 
dimensions : 416, 956, 397696  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 507752.5, 555552.5, 8409803, 8430603  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_rugosity/tut_rug.grd 
names      : Tutuila_5m_Rugosity 
values     : 1.000075, 9.170565  (min, max)


class      : RasterLayer 
dimensions : 417, 956, 398652  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 507747.5, 555547.5, 8409758, 8430608  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_slope/tut_slope.grd 
names      : Tutuila_5m_Slope 
values     : 0, 72.97091  (min, max)


class      : RasterLayer 
dimensions : 417, 956, 398652  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 507747.5, 555547.5, 8409758, 8430608  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_asp/tut_aspect.grd 
names      : aspect 
values     : 2.544444e-12, 358.727  (min, max)


Updating layer `tut_TOAD' to data source `C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD' using driver `ESRI Shapefile'
Updating existing layer tut_TOAD
Writing 5602 features with 107 fields and geometry type Point.


## Rose Atoll

In [9]:
rose_bty <-raster('Rose/Rose_bty/rose_5m_dbmb.asc')
rose_hs <- raster('Rose/Rose_hs/ros_5m_hs_int.asc')
rose_rug <- raster('Rose/Rose_rugosity/ros_5m_rug.asc')
rose_slope <- raster('Rose/Rose_slope/ros_5m_slp.asc')

proj4string(rose_bty) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
proj4string(rose_hs) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
proj4string(rose_rug) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
proj4string(rose_slope) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")

rose_aspect <- terrain(rose_bty, opt = "aspect", unit = "degrees")

#### Clip bathymetry raster < 30 m, > 200 m; crop others to same extent; resample rasters

In [10]:
rose_bty[rose_bty > -30] <- NA
rose_bty[rose_bty < -200] <- NA

rose_hs <- crop(rose_hs, rose_bty)
rose_rug <- crop(rose_rug, rose_bty)
rose_slope <- crop(rose_slope, rose_bty)
rose_aspect <- crop(rose_aspect, rose_bty)

rose_bty <- aggregate(rose_bty, fact=10)
rose_hs <- aggregate(rose_hs, fact=10)
rose_rug <- aggregate(rose_rug, fact=10)
rose_slope <- aggregate(rose_slope, fact=10)
rose_aspect <- aggregate(rose_aspect, fact=10)

In [11]:
# Write tranformed rasters
writeRaster(rose_bty, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Rose/Rose_bty',
                                       "rose_bty.grd"), format='raster', overwrite=TRUE)
writeRaster(rose_hs, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Rose/Rose_hs',
                                       "rose_hs.grd"), format='raster', overwrite=TRUE)
writeRaster(rose_rug, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Rose/Rose_rugosity',
                                       "rose_rug.grd"), format='raster', overwrite=TRUE)
writeRaster(rose_slope, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Rose/Rose_slope',
                                       "rose_slope.grd"), format='raster', overwrite=TRUE)
writeRaster(rose_aspect, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Rose/Rose_asp',
                                       "rose_aspect.grd"), format='raster', overwrite=TRUE)

class      : RasterLayer 
dimensions : 84, 92, 7728  (nrow, ncol, ncell)
resolution : 50.0109, 50.0109  (x, y)
extent     : 804400, 809001, 8388065, 8392266  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Rose/Rose_bty/rose_bty.grd 
names      : rose_5m_dbmb 
values     : -199.771, -30.1904  (min, max)


class      : RasterLayer 
dimensions : 84, 92, 7728  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 804398.2, 808998.2, 8388065, 8392265  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Rose/Rose_hs/rose_hs.grd 
names      : ros_5m_hs_int 
values     : 1, 2  (min, max)


class      : RasterLayer 
dimensions : 84, 92, 7728  (nrow, ncol, ncell)
resolution : 50.0109, 50.0109  (x, y)
extent     : 804405, 809006, 8388060, 8392261  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Rose/Rose_rugosity/rose_rug.grd 
names      : ros_5m_rug 
values     : 1.000326, 6.405884  (min, max)


class      : RasterLayer 
dimensions : 84, 92, 7728  (nrow, ncol, ncell)
resolution : 50.0109, 50.0109  (x, y)
extent     : 804400, 809001, 8388065, 8392266  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Rose/Rose_slope/rose_slope.grd 
names      : ros_5m_slp 
values     : 0.1387146, 76.21767  (min, max)


class      : RasterLayer 
dimensions : 84, 92, 7728  (nrow, ncol, ncell)
resolution : 50.0109, 50.0109  (x, y)
extent     : 804400, 809001, 8388065, 8392266  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Rose/Rose_asp/rose_aspect.grd 
names      : aspect 
values     : 21.63174, 351.2603  (min, max)


## Swains Island

In [12]:
swains_bty <-raster('Swains/Swains_bty/swa_dbmb_5m.asc')
swains_hs <- raster('Swains/Swains_hs/swains_40m_hardsoft.asc')
swains_rug <- raster('Swains/Swains_rugosity/swa_40m_rug.asc')
swains_slope <- raster('Swains/Swains_slope/swa_40m_slp.asc')

proj4string(swains_bty) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
proj4string(swains_hs) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
proj4string(swains_rug) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
proj4string(swains_slope) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")

swains_aspect <- terrain(swains_bty, opt = "aspect", unit = "degrees")

#### Clip bathymetry raster < 30 m, > 200 m; crop others to same extent; resample rasters

In [13]:
swains_bty[swains_bty > -30] <- NA
swains_bty[swains_bty < -200] <- NA

swains_hs <- crop(swains_hs, swains_bty)
swains_rug <- crop(swains_rug, swains_bty)
swains_slope <- crop(swains_slope, swains_bty)
swains_aspect <- crop(swains_aspect, swains_bty)

swains_bty <- aggregate(swains_bty, fact=10)

swains_hs <- disaggregate(swains_hs, fact=4)
swains_hs <- aggregate(swains_hs, fact=5)

swains_rug <- disaggregate(swains_rug, fact=4)
swains_rug <- aggregate(swains_rug, fact=5)

swains_slope <- disaggregate(swains_slope, fact=4)
swains_slope <- aggregate(swains_slope, fact=5)

swains_aspect <- aggregate(swains_aspect, fact=10)

In [14]:
# Write tranformed rasters and shp to file
writeRaster(swains_bty, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Swains/Swains_bty',
                                       "swains_bty.grd"), format='raster', overwrite=TRUE)
writeRaster(swains_hs, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Swains/Swains_hs',
                                       "swains_hs.grd"), format='raster', overwrite=TRUE)
writeRaster(swains_rug, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Swains/Swains_rugosity',
                                       "swains_rug.grd"), format='raster', overwrite=TRUE)
writeRaster(swains_slope, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Swains/Swains_slope',
                                       "swains_slope.grd"), format='raster', overwrite=TRUE)
writeRaster(swains_aspect, filename=file.path('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Swains/Swains_asp',
                                       "swains_aspect.grd"), format='raster', overwrite=TRUE)

class      : RasterLayer 
dimensions : 61, 73, 4453  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 489660.1, 493310.1, 8776165, 8779215  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Swains/Swains_bty/swains_bty.grd 
names      : swa_dbmb_5m 
values     : -196.91, -30.3  (min, max)


class      : RasterLayer 
dimensions : 60, 72, 4320  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 489655, 493255, 8776205, 8779205  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Swains/Swains_hs/swains_hs.grd 
names      : swains_40m_hardsoft 
values     : 2, 2  (min, max)


class      : RasterLayer 
dimensions : 60, 72, 4320  (nrow, ncol, ncell)
resolution : 49.97388, 49.97387  (x, y)
extent     : 489668.5, 493266.6, 8776213, 8779211  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Swains/Swains_rugosity/swains_rug.grd 
names      : swa_40m_rug 
values     : 1.105439, 2.955958  (min, max)


class      : RasterLayer 
dimensions : 60, 72, 4320  (nrow, ncol, ncell)
resolution : 49.97388, 49.97387  (x, y)
extent     : 489668.5, 493266.6, 8776213, 8779211  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Swains/Swains_slope/swains_slope.grd 
names      : swa_40m_slp 
values     : 0.5114931, 69.53071  (min, max)


class      : RasterLayer 
dimensions : 61, 73, 4453  (nrow, ncol, ncell)
resolution : 50, 50  (x, y)
extent     : 489660.1, 493310.1, 8776165, 8779215  (xmin, xmax, ymin, ymax)
crs        : +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs 
source     : C:/Users/linds/OneDrive/Documents/samoa_corals_data/Swains/Swains_asp/swains_aspect.grd 
names      : aspect 
values     : 12.72434, 345.2911  (min, max)


## American Samoa (all)
Load the u (eastward) currents nc file first:

In [15]:
setwd('C:/Users/linds/OneDrive/Documents/samoa_corals_data/')
currents <- nc_open('AS_all/Currents/currents.nc')
#str(currents)

In [16]:
v1 = currents$var[[1]] 
u = ncvar_get(currents, v1) # 2929
lon = v1$dim[[1]]$vals # dim[[1]] is lon - 186
lat = v1$dim[[2]]$vals # dim[[2]] is lat - 112

# Using deconstruct function c() to turn matrices into numeric class output
uvals <- c(u) # length = 61016928, which is u*lon*lat
df <- as.data.frame(uvals)

# Same lat applied to 186 uvals at a time (there are 186 lon values before they repeat)
latlist <- rep(lat,each=186)
# 1 column and 61016928 (112*186*2929) rows in uvel 
colnames(df) <- "u"
lonlist <- rep(lon, times=112)
df$lon <- lonlist
df$lat <- latlist

d <- split(df, rep(1:2929, each=20832))
merged <- d %>% reduce(left_join, by=c("lat", "lon"), all=FALSE)
udf <- data.frame(lon=merged[,2], lat=merged[,3], umean=rowMeans(merged[,-c(2,3)]))

Next, load the v (westward) currents:

In [17]:
v3 = currents$var[[3]] 
v = ncvar_get(currents, v3)
lon = v3$dim[[1]]$vals # dim[[1]] is lon - 186
lat = v3$dim[[2]]$vals # dim[[2]] is lat - 112
vvals <- c(v) # length = 61016928, which is v*lon*lat
df2 <- as.data.frame(vvals)
latlist <- rep(lat,each=186) 
colnames(df) <- "v"
lonlist <- rep(lon, times=112)
df2$lon <- lonlist
df2$lat <- latlist


d2 <- split(df2, rep(1:2929, each=20832))
merged2 <- d2 %>% reduce(left_join, by=c("lat", "lon"), all=FALSE)
vdf <- data.frame(lon=merged2[,2], lat=merged2[,3], vmean=rowMeans(merged2[,-c(2,3)]))

# Write CSVs to file
udf <- na.omit(udf)
vdf <- na.omit(vdf)
write.csv(udf, 'AS_all/Currents/udf.csv', row.names = FALSE)
write.csv(vdf, 'AS_all/Currents/vdf.csv', row.names = FALSE)

In [18]:
# Reload a "raw", untransformed raster
tut_slope_raw <- raster('Tut/Tut_slope/Tutuila_5m_Slope.asc')
proj4string(tut_slope_raw) <- CRS("+proj=longlat +datum=WGS84 +no_defs")
crs(tut_slope_raw)

# Create CRS objects
utm_crs <- crs(tut_bty)
wgs_crs <- crs(tut_slope_raw)

# Assign the original CRS to a new SpatialPointsDataFrame object
udf_sp <- SpatialPointsDataFrame(udf[,1:2], udf, proj4string = wgs_crs)
vdf_sp <- SpatialPointsDataFrame(vdf[,1:2], vdf, proj4string = wgs_crs)

# Check CRS
#crs(udf_sp)
#extent(udf_sp)

# Write as a shapefile
writeOGR(obj=udf_sp, layer="umean", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/AS_all/Currents', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=vdf_sp, layer="vmean", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/AS_all/Currents', driver="ESRI Shapefile",
         overwrite_layer=TRUE)

# Reload and transform
ucur <- st_read('AS_all/Currents/umean.shp')
ucur <- st_transform(ucur, crs= st_crs(tut_bty))
vcur <- st_read('AS_all/Currents/vmean.shp')
vcur <- st_transform(vcur, crs= st_crs(tut_bty))

# Write as a tranformed shapefile
st_write(obj=ucur, dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/AS_all/Currents', 
         update=TRUE, driver="ESRI Shapefile", 'ucur.shp')
st_write(obj=vcur, dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/AS_all/Currents', 
         update=TRUE, driver="ESRI Shapefile", 'vcur.shp')

CRS arguments:
 +proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0 

Reading layer `umean' from data source `C:\Users\linds\OneDrive\Documents\samoa_corals_data\AS_all\Currents\umean.shp' using driver `ESRI Shapefile'
Simple feature collection with 20359 features and 3 fields
geometry type:  POINT
dimension:      XY
bbox:           xmin: -174 ymin: -15.5 xmax: -168.8088 ymax: -12.49855
epsg (SRID):    4326
proj4string:    +proj=longlat +datum=WGS84 +no_defs
Reading layer `vmean' from data source `C:\Users\linds\OneDrive\Documents\samoa_corals_data\AS_all\Currents\vmean.shp' using driver `ESRI Shapefile'
Simple feature collection with 20359 features and 3 fields
geometry type:  POINT
dimension:      XY
bbox:           xmin: -174 ymin: -15.5 xmax: -168.8088 ymax: -12.49855
epsg (SRID):    4326
proj4string:    +proj=longlat +datum=WGS84 +no_defs
Updating layer `ucur' to data source `C:/Users/linds/OneDrive/Documents/samoa_corals_data/AS_all/Currents' using driver `ESRI Shapefile'
Updating existing layer ucur
Writing 20359 features with 3 fields and geometr

Load the remainder of the data:

In [19]:
# setwd('C:/Users/linds/OneDrive/Documents/samoa_corals_data/')
ucur <- st_read('AS_all/Currents/ucur.shp')
vcur <- st_read('AS_all/Currents/vcur.shp')

esi <- st_read('AS_all/Env_sens_index/esi_raw.shp')
pop <- st_read('AS_all/Population/pop_raw.shp')
shore <- st_read('AS_all/Shorelines/shore_raw.shp')
villages <- st_read('AS_all/Villages/villages_raw.shp')
wetlands <- st_read('AS_all/Wetlands/wl_raw.shp')

esi <- st_transform(esi, crs= st_crs(tut_bty))
pop <- st_transform(pop, crs= st_crs(tut_bty))
shore <- st_transform(shore, crs= st_crs(tut_bty))
villages <- st_transform(villages, crs= st_crs(tut_bty))
wetlands <- st_transform(wetlands, crs= st_crs(tut_bty))
# Very finicky layer with some troublesome data types
wetlands$ID <- NULL
wetlands$LAYER <- NULL
wetlands$VILLAGE <- NULL
wetlands$NAME <- NULL
wetlands$SHP_SOURCE <- NULL
wetlands$SOURCE <- NULL

# Write as a tranformed shapefile
st_write(obj=esi, dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/AS_all/Env_sens_index', 
         update=TRUE, driver="ESRI Shapefile", 'esi.shp')
st_write(obj=pop, dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/AS_all/Population', 
         update=TRUE, driver="ESRI Shapefile", 'pop.shp')
st_write(obj=shore, dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/AS_all/Shorelines', 
         update=TRUE, driver="ESRI Shapefile", 'shore.shp')
st_write(obj=villages, dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/AS_all/Villages', 
         update=TRUE, driver="ESRI Shapefile", 'villages.shp')
st_write(obj=wetlands, dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/AS_all/Wetlands', 
         update=TRUE, driver="ESRI Shapefile", 'wetlands.shp')

Reading layer `ucur' from data source `C:\Users\linds\OneDrive\Documents\samoa_corals_data\AS_all\Currents\ucur.shp' using driver `ESRI Shapefile'
Simple feature collection with 61077 features and 3 fields
geometry type:  POINT
dimension:      XY
bbox:           xmin: 173899.2 ymin: 8284117 xmax: 738140.2 ymax: 8618317
epsg (SRID):    NA
proj4string:    +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs
Reading layer `vcur' from data source `C:\Users\linds\OneDrive\Documents\samoa_corals_data\AS_all\Currents\vcur.shp' using driver `ESRI Shapefile'
Simple feature collection with 61077 features and 3 fields
geometry type:  POINT
dimension:      XY
bbox:           xmin: 173899.2 ymin: 8284117 xmax: 738140.2 ymax: 8618317
epsg (SRID):    NA
proj4string:    +proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs
Reading layer `esi_raw' from data source `C:\Users\linds\OneDrive\Documents\samoa_corals_data\AS_all\Env_sens_index\esi_raw.shp' using driver `ESRI Shapefile'
Simple feature co

Warning message in CPL_write_ogr(obj, dsn, layer, driver, as.character(dataset_options), :
"GDAL Message 1: Value 135995000 of field AREA of feature 2129 not successfully written. Possibly due to too larger number with respect to field width"


Updating layer `villages' to data source `C:/Users/linds/OneDrive/Documents/samoa_corals_data/AS_all/Villages' using driver `ESRI Shapefile'
Updating existing layer villages
Writing 8 features with 5 fields and geometry type Multi Polygon.
Updating layer `wetlands' to data source `C:/Users/linds/OneDrive/Documents/samoa_corals_data/AS_all/Wetlands' using driver `ESRI Shapefile'
Updating existing layer wetlands
Writing 22 features with 3 fields and geometry type Polygon.


## Create dataframe for model training

I need to...
- [done] clean up shapefiles, i.e. ensure each 50 m cell only has a single observation; rasterize, then reconvert to points shapefile
- [done] extract data from rasters to points
- [done] calculate Euclidean distance for:
    - Villages > 3rd quantile (from PacIOOS: "AS_all/Villages/")
    - Distance from shore (from PacIOOS: "AS_all/Shorelines/")
    - Wetland proximity (from PacIOOS: "AS_all/Wetlands/")
- [done] extract closest values for:
    - Environmental sensitivity index (from PacIOOS: "AS_all/Env_sens_index")
    - ROMS u velocity (to E) (3 km resolution; from PacIOOS: "AS_all/Currents")
    - ROMS v velocity (to N) (3 km resolution; from PacIOOS: "AS_all/Currents")
- impute missing data *(will get to this when I make group-specific dfs)*


The cell below should be run in a new session to reload the clean layers:

In [24]:
setwd("C:/Users/linds/OneDrive/Documents/samoa_corals_data/")

library(data.table)
library(geojsonio)
library(GISTools)
library(ncdf4)
library(raster)
library(rgdal)
library(rgeos)
library(sf)
library(spatial.tools)
library(tidyverse)

# Ofu-Olo
oo_bty <-raster('OfuOlo/OfuOlo_bty/oo_bty.grd')
oo_hs <- raster('OfuOlo/OfuOlo_hs/oo_hs.grd')
oo_rug <- raster('OfuOlo/OfuOlo_rugosity/oo_rug.grd')
oo_slope <- raster('OfuOlo/OfuOlo_slope/oo_slope.grd')
oo_asp <- raster('OfuOlo/OfuOlo_asp/oo_aspect.grd')
oo_TOAD <- readOGR('OfuOlo/OfuOlo_TOAD/oo_TOAD.shp') 

# Tau
tau_bty <-raster('Tau/Tau_bty/tau_bty.grd')
tau_hs <- raster('Tau/Tau_hs/tau_hs.grd')
tau_rug <- raster('Tau/Tau_rugosity/tau_rug.grd')
tau_slope <- raster('Tau/Tau_slope/tau_slope.grd')
tau_asp <- raster('Tau/Tau_asp/tau_aspect.grd')
tau_TOAD <- readOGR('Tau/Tau_TOAD/tau_TOAD.shp') 

# Tut
tut_bty <-raster('Tut/tut_bty/tut_bty.grd')
tut_hs <- raster('Tut/tut_hs/tut_hs.grd')
tut_rug <- raster('Tut/tut_rugosity/tut_rug.grd')
tut_slope <- raster('Tut/tut_slope/tut_slope.grd')
tut_asp <- raster('Tut/tut_asp/tut_aspect.grd')
tut_TOAD <- readOGR('Tut/tut_TOAD/tut_TOAD.shp') 

# Rose 
rose_bty <-raster('Rose/rose_bty/rose_bty.grd')
rose_hs <- raster('Rose/rose_hs/rose_hs.grd')
rose_rug <- raster('Rose/rose_rugosity/rose_rug.grd')
rose_slope <- raster('Rose/rose_slope/rose_slope.grd')
rose_asp <- raster('Rose/rose_asp/rose_aspect.grd')

# Swains
swains_bty <-raster('Swains/swains_bty/swains_bty.grd')
swains_hs <- raster('Swains/swains_hs/swains_hs.grd')
swains_rug <- raster('Swains/swains_rugosity/swains_rug.grd')
swains_slope <- raster('Swains/swains_slope/swains_slope.grd')
swains_asp <- raster('Swains/swains_asp/swains_aspect.grd')

# All
ucur <- readOGR('AS_all/Currents/ucur.shp')
vcur <- readOGR('AS_all/Currents/vcur.shp')
esi <- readOGR('AS_all/Env_sens_index/esi.shp')
shore <- readOGR('AS_all/Shorelines/shore.shp')
villages <- readOGR('AS_all/Villages/villages.shp')
wetlands <- readOGR('AS_all/Wetlands/wetlands.shp')

OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\linds\OneDrive\Documents\samoa_corals_data\OfuOlo\OfuOlo_TOAD\oo_TOAD.shp", layer: "oo_TOAD"
with 18252 features
It has 97 fields
OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\linds\OneDrive\Documents\samoa_corals_data\Tau\Tau_TOAD\tau_TOAD.shp", layer: "tau_TOAD"
with 5104 features
It has 97 fields
OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\linds\OneDrive\Documents\samoa_corals_data\Tut\Tut_TOAD\tut_TOAD.shp", layer: "tut_TOAD"
with 33612 features
It has 107 fields
OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\linds\OneDrive\Documents\samoa_corals_data\AS_all\Currents\ucur.shp", layer: "ucur"
with 61077 features
It has 3 fields
OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\linds\OneDrive\Documents\samoa_corals_data\AS_all\Currents\vcur.shp", layer: "vcur"
with 61077 features
It has 3 fields
OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\

#### Rasterize TOAD shapefiles
For each coral grouping, an unknown number of 50 m cells have multiple, possibly conflicting, observations. With this protocol, the 1s are preserved within the resulting raster.

1. [done] SCLER_CORA: Tut
2. [done] MASSIVE: OfuOlo, Tau, Tut
3. [done] PLATE_LIKE: Tut
4. [done] ENCRUSTING: OfuOlo, Tau, Tut
5. [done] BRANCHING: OfuOlo, Tau, Tut
6. [done] COLUMNAR: OfuOlo, Tau, Tut
7. [done] FREE_LIVIN: OfuOlo, Tau, Tut
    
Rose and Swains don't have observational data, so I need to predict to those raster bricks, but can't train a model to them.

In [26]:
# Create CRS object
utm_crs <- crs(tut_bty)

# Set up regional raster "templates" to use in rasterize()
## Ofu-Olo
oo_extent <-  extent(oo_bty)
xy <- abs(apply(as.matrix(bbox(oo_extent)), 1, diff))
n <- 5
r <- raster(oo_extent, ncol=xy[1]*n, nrow=xy[2]*n)
proj4string(r) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
# res(r) is currently 0.2; resample raster to a resolution of 50 m (factor of 250)
oo_r <- aggregate(r, fact=250)

## Tau
tau_extent <-  extent(tau_bty)
xy <- abs(apply(as.matrix(bbox(tau_extent)), 1, diff))
n <- 5
r <- raster(tau_extent, ncol=xy[1]*n, nrow=xy[2]*n)
proj4string(r) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
# res(r) is currently 0.2; resample raster to a resolution of 50 m (factor of 250)
tau_r <- aggregate(r, fact=250)

## Tut
tut_extent <-  extent(tut_bty)
xy <- abs(apply(as.matrix(bbox(tut_extent)), 1, diff))
n <- 5
r <- raster(tut_extent, ncol=xy[1]*n, nrow=xy[2]*n)
proj4string(r) <- CRS("+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs")
# res(r) is currently 0.2; resample raster to a resolution of 50 m (factor of 250)
tut_r <- aggregate(r, fact=250)

#### SCLER_CORA

In [27]:
## Tut

# Rasterize the shapefile
tut_scler_r <- rasterize(tut_TOAD[83], tut_r)

#writeRaster(tut_scler_r, "tut_scler_r.grd")
#plot(tut_scler_r)

# Convert raster to shp
tut_scler_pts <- rasterToPoints(tut_scler_r, spatial=TRUE)

# Convert to df
tut_scler_df <- as.data.frame(tut_scler_pts)
# Change name for clarity
names(tut_scler_df)[names(tut_scler_df) == 'SCLER_CORA'] = 'Scler_Raw'
# Loop through to populate the binary column
for (i in 1:nrow(tut_scler_df)) {
    if (tut_scler_df$Scler_Raw[i] == 0) {
        tut_scler_df$Scler_Coral[i] = 0}  
    else { 
        tut_scler_df$Scler_Coral[i] = 1
}}

# Reconvert to shp...
# Assign the UTM CRS to a new SpatialPointsDataFrame object
coords <- (tut_scler_df[3:4])
tut_scler_sp <- SpatialPointsDataFrame(coords, tut_scler_df, proj4string = utm_crs)

# Write as a shapefile
writeOGR(obj=tut_scler_sp, layer="Scler_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=tut_scler_sp, layer="Scler_Raw", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)

Warning message in writeOGR(obj = tut_scler_sp, layer = "Scler_Coral", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = tut_scler_sp, layer = "Scler_Raw", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"


#### MASSIVE

In [28]:
## Tau
tau_massive_r <- rasterize(tau_TOAD[64], tau_r)
tau_massive_pts <- rasterToPoints(tau_massive_r, spatial=TRUE)
tau_massive_df <- as.data.frame(tau_massive_pts)
names(tau_massive_df)[names(tau_massive_df) == 'Massive'] = 'Massive_Raw'
# Loop through to populate the binary column
for (i in 1:nrow(tau_massive_df)) {
    if (tau_massive_df$Massive_Raw[i] == 0) {
        tau_massive_df$Massive_Coral[i] = 0}  
    else { 
        tau_massive_df$Massive_Coral[i] = 1
}}
coords <- (tau_massive_df[3:4])
tau_massive_sp <- SpatialPointsDataFrame(coords, tau_massive_df, proj4string = utm_crs)
writeOGR(obj=tau_massive_sp, layer="Massive_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=tau_massive_sp, layer="Massive_Raw", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)

Warning message in writeOGR(obj = tau_massive_sp, layer = "Massive_Coral", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = tau_massive_sp, layer = "Massive_Raw", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"


In [29]:
## Tut
tut_massive_r <- rasterize(tut_TOAD[101], tut_r)
tut_massive_pts <- rasterToPoints(tut_massive_r, spatial=TRUE)
tut_massive_df <- as.data.frame(tut_massive_pts)
names(tut_massive_df)[names(tut_massive_df) == 'MASSIVE'] = 'Massive_Raw'
for (i in 1:nrow(tut_massive_df)) {
    if (tut_massive_df$Massive_Raw[i] == 0) {
        tut_massive_df$Massive_Coral[i] = 0}  
    else { 
        tut_massive_df$Massive_Coral[i] = 1
}}
coords <- (tut_massive_df[3:4])
tut_massive_sp <- SpatialPointsDataFrame(coords, tut_massive_df, proj4string = utm_crs)
writeOGR(obj=tut_massive_sp, layer="Massive_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=tut_massive_sp, layer="Massive_Raw", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)

Warning message in writeOGR(obj = tut_massive_sp, layer = "Massive_Coral", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = tut_massive_sp, layer = "Massive_Raw", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"


In [30]:
## Ofu-Olo
oo_massive_r <- rasterize(oo_TOAD[64], oo_r)
oo_massive_pts <- rasterToPoints(oo_massive_r, spatial=TRUE)
oo_massive_df <- as.data.frame(oo_massive_pts)
names(oo_massive_df)[names(oo_massive_df) == 'Massive'] = 'Massive_Raw'
# Loop through to populate the binary column
for (i in 1:nrow(oo_massive_df)) {
    if (oo_massive_df$Massive_Raw[i] == 0) {
        oo_massive_df$Massive_Coral[i] = 0}  
    else { 
        oo_massive_df$Massive_Coral[i] = 1
}}
coords <- (oo_massive_df[3:4])
oo_massive_sp <- SpatialPointsDataFrame(coords, oo_massive_df, proj4string = utm_crs)
writeOGR(obj=oo_massive_sp, layer="Massive_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=oo_massive_sp, layer="Massive_Raw", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)

Warning message in writeOGR(obj = oo_massive_sp, layer = "Massive_Coral", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = oo_massive_sp, layer = "Massive_Raw", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"


#### PLATE_LIKE

In [31]:
## Tut
tut_plate_like_r <- rasterize(tut_TOAD[102],tut_r)
tut_plate_like_pts <- rasterToPoints(tut_plate_like_r, spatial=TRUE)
tut_plate_like_df <- as.data.frame(tut_plate_like_pts)
names(tut_plate_like_df)[names(tut_plate_like_df) == 'PLATE_LIKE'] = 'Plate_Like_Raw'
for (i in 1:nrow(tut_plate_like_df)) {
    if (tut_plate_like_df$Plate_Like_Raw[i] == 0) {
        tut_plate_like_df$Plate_Like_Coral[i] = 0}  
    else { 
        tut_plate_like_df$Plate_Like_Coral[i] = 1
}}
coords <- (tut_plate_like_df[3:4])
tut_plate_like_sp <- SpatialPointsDataFrame(coords, tut_plate_like_df, proj4string = utm_crs)
writeOGR(obj=tut_plate_like_sp, layer="Plate_Like_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)

Warning message in writeOGR(obj = tut_plate_like_sp, layer = "Plate_Like_Coral", :
"Field names abbreviated for ESRI Shapefile driver"


#### ENCRUSTING

In [32]:
## Tau
tau_encrusting_r <- rasterize(tau_TOAD[61], tau_r)
tau_encrusting_pts <- rasterToPoints(tau_encrusting_r, spatial=TRUE)
tau_encrusting_df <- as.data.frame(tau_encrusting_pts)
names(tau_encrusting_df)[names(tau_encrusting_df) == 'Encrusting'] = 'Encrusting_Raw'
# Loop through to populate the binary column
for (i in 1:nrow(tau_encrusting_df)) {
    if (tau_encrusting_df$Encrusting_Raw[i] == 0) {
        tau_encrusting_df$Encrusting_Coral[i] = 0}  
    else { 
        tau_encrusting_df$Encrusting_Coral[i] = 1
}}
coords <- (tau_encrusting_df[3:4])
tau_encrusting_sp <- SpatialPointsDataFrame(coords, tau_encrusting_df, proj4string = utm_crs)
writeOGR(obj=tau_encrusting_sp, layer="Encrusting_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=tau_encrusting_sp, layer="Encrusting_Raw", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)


## Tut
tut_encrusting_r <- rasterize(tut_TOAD[103], tut_r)
tut_encrusting_pts <- rasterToPoints(tut_encrusting_r, spatial=TRUE)
tut_encrusting_df <- as.data.frame(tut_encrusting_pts)
names(tut_encrusting_df)[names(tut_encrusting_df) == 'ENCRUSTING'] = 'Encrusting_Raw'
for (i in 1:nrow(tut_encrusting_df)) {
    if (tut_encrusting_df$Encrusting_Raw[i] == 0) {
        tut_encrusting_df$Encrusting_Coral[i] = 0}  
    else { 
        tut_encrusting_df$Encrusting_Coral[i] = 1
}}
coords <- (tut_encrusting_df[3:4])
tut_encrusting_sp <- SpatialPointsDataFrame(coords, tut_encrusting_df, proj4string = utm_crs)
writeOGR(obj=tut_encrusting_sp, layer="Encrusting_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=tut_encrusting_sp, layer="Encrusting_Raw", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)


## Ofu-Olo
oo_encrusting_r <- rasterize(oo_TOAD[61], oo_r)
oo_encrusting_pts <- rasterToPoints(oo_encrusting_r, spatial=TRUE)
oo_encrusting_df <- as.data.frame(oo_encrusting_pts)
names(oo_encrusting_df)[names(oo_encrusting_df) == 'Encrusting'] = 'Encrusting_Raw'
# Loop through to populate the binary column
for (i in 1:nrow(oo_encrusting_df)) {
    if (oo_encrusting_df$Encrusting_Raw[i] == 0) {
        oo_encrusting_df$Encrusting_Coral[i] = 0}  
    else { 
        oo_encrusting_df$Encrusting_Coral[i] = 1
}}
coords <- (oo_encrusting_df[3:4])
oo_encrusting_sp <- SpatialPointsDataFrame(coords, oo_encrusting_df, proj4string = utm_crs)
writeOGR(obj=oo_encrusting_sp, layer="Encrusting_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=oo_encrusting_sp, layer="Encrusting_Raw", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)

Warning message in writeOGR(obj = tau_encrusting_sp, layer = "Encrusting_Coral", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = tau_encrusting_sp, layer = "Encrusting_Raw", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = tut_encrusting_sp, layer = "Encrusting_Coral", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = tut_encrusting_sp, layer = "Encrusting_Raw", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = oo_encrusting_sp, layer = "Encrusting_Coral", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = oo_encrusting_sp, layer = "Encrusting_Raw", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD", :
"Field names abbreviat

#### BRANCHING

In [33]:
## Tau
tau_branching_r <- rasterize(tau_TOAD[59], tau_r)
tau_branching_pts <- rasterToPoints(tau_branching_r, spatial=TRUE)
tau_branching_df <- as.data.frame(tau_branching_pts)
names(tau_branching_df)[names(tau_branching_df) == 'Branching'] = 'Branching_Raw'
# Loop through to populate the binary column
for (i in 1:nrow(tau_branching_df)) {
    if (tau_branching_df$Branching_Raw[i] == 0) {
        tau_branching_df$Branching_Coral[i] = 0}  
    else { 
        tau_branching_df$Branching_Coral[i] = 1
}}
coords <- (tau_branching_df[3:4])
tau_branching_sp <- SpatialPointsDataFrame(coords, tau_branching_df, proj4string = utm_crs)
writeOGR(obj=tau_branching_sp, layer="Branching_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=tau_branching_sp, layer="Branching_Raw", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)

## Tut
tut_branching_r <- rasterize(tut_TOAD[104], tut_r)
tut_branching_pts <- rasterToPoints(tut_branching_r, spatial=TRUE)
tut_branching_df <- as.data.frame(tut_branching_pts)
names(tut_branching_df)[names(tut_branching_df) == 'BRANCHING'] = 'Branching_Raw'
for (i in 1:nrow(tut_branching_df)) {
    if (tut_branching_df$Branching_Raw[i] == 0) {
        tut_branching_df$Branching_Coral[i] = 0}  
    else { 
        tut_branching_df$Branching_Coral[i] = 1
}}
coords <- (tut_branching_df[3:4])
tut_branching_sp <- SpatialPointsDataFrame(coords, tut_branching_df, proj4string = utm_crs)
writeOGR(obj=tut_branching_sp, layer="Branching_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=tut_branching_sp, layer="Branching_Raw", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)

## Ofu-Olo
oo_branching_r <- rasterize(oo_TOAD[59], oo_r)
oo_branching_pts <- rasterToPoints(oo_branching_r, spatial=TRUE)
oo_branching_df <- as.data.frame(oo_branching_pts)
names(oo_branching_df)[names(oo_branching_df) == 'Branching'] = 'Branching_Raw'
# Loop through to populate the binary column
for (i in 1:nrow(oo_branching_df)) {
    if (oo_branching_df$Branching_Raw[i] == 0) {
        oo_branching_df$Branching_Coral[i] = 0}  
    else { 
        oo_branching_df$Branching_Coral[i] = 1
}}
coords <- (oo_branching_df[3:4])
oo_branching_sp <- SpatialPointsDataFrame(coords, oo_branching_df, proj4string = utm_crs)
writeOGR(obj=oo_branching_sp, layer="Branching_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=oo_branching_sp, layer="Branching_Raw", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)

Warning message in writeOGR(obj = tau_branching_sp, layer = "Branching_Coral", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = tau_branching_sp, layer = "Branching_Raw", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = tut_branching_sp, layer = "Branching_Coral", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = tut_branching_sp, layer = "Branching_Raw", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = oo_branching_sp, layer = "Branching_Coral", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD", :
"Field names abb

#### COLUMNAR

In [34]:
## Tau
tau_columnar_r <- rasterize(tau_TOAD[60], tau_r)
tau_columnar_pts <- rasterToPoints(tau_columnar_r, spatial=TRUE)
tau_columnar_df <- as.data.frame(tau_columnar_pts)
names(tau_columnar_df)[names(tau_columnar_df) == 'Columnar'] = 'Columnar_Raw'
# Loop through to populate the binary column
for (i in 1:nrow(tau_columnar_df)) {
    if (tau_columnar_df$Columnar_Raw[i] == 0) {
        tau_columnar_df$Columnar_Coral[i] = 0}  
    else { 
        tau_columnar_df$Columnar_Coral[i] = 1
}}
coords <- (tau_columnar_df[3:4])
tau_columnar_sp <- SpatialPointsDataFrame(coords, tau_columnar_df, proj4string = utm_crs)
writeOGR(obj=tau_columnar_sp, layer="Columnar_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=tau_columnar_sp, layer="Columnar_Raw", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)


## Tut
tut_columnar_r <- rasterize(tut_TOAD[105], tut_r)
tut_columnar_pts <- rasterToPoints(tut_columnar_r, spatial=TRUE)
tut_columnar_df <- as.data.frame(tut_columnar_pts)
names(tut_columnar_df)[names(tut_columnar_df) == 'COLUMNAR'] = 'Columnar_Raw'
for (i in 1:nrow(tut_columnar_df)) {
    if (tut_columnar_df$Columnar_Raw[i] == 0) {
        tut_columnar_df$Columnar_Coral[i] = 0}  
    else { 
        tut_columnar_df$Columnar_Coral[i] = 1
}}
coords <- (tut_columnar_df[3:4])
tut_columnar_sp <- SpatialPointsDataFrame(coords, tut_columnar_df, proj4string = utm_crs)
writeOGR(obj=tut_columnar_sp, layer="Columnar_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=tut_columnar_sp, layer="Columnar_Raw", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)


## Ofu-Olo
oo_columnar_r <- rasterize(oo_TOAD[60], oo_r)
oo_columnar_pts <- rasterToPoints(oo_columnar_r, spatial=TRUE)
oo_columnar_df <- as.data.frame(oo_columnar_pts)
names(oo_columnar_df)[names(oo_columnar_df) == 'Columnar'] = 'Columnar_Raw'
# Loop through to populate the binary column
for (i in 1:nrow(oo_columnar_df)) {
    if (oo_columnar_df$Columnar_Raw[i] == 0) {
        oo_columnar_df$Columnar_Coral[i] = 0}  
    else { 
        oo_columnar_df$Columnar_Coral[i] = 1
}}
coords <- (oo_columnar_df[3:4])
oo_columnar_sp <- SpatialPointsDataFrame(coords, oo_columnar_df, proj4string = utm_crs)
writeOGR(obj=oo_columnar_sp, layer="Columnar_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=oo_columnar_sp, layer="Columnar_Raw", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)

Warning message in writeOGR(obj = tau_columnar_sp, layer = "Columnar_Coral", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = tau_columnar_sp, layer = "Columnar_Raw", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = tut_columnar_sp, layer = "Columnar_Coral", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = tut_columnar_sp, layer = "Columnar_Raw", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = oo_columnar_sp, layer = "Columnar_Coral", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD", :
"Field names abbreviated f

#### FREE_LIVIN

In [35]:
## Tau
tau_free_livin_r <- rasterize(tau_TOAD[62], tau_r)
tau_free_livin_pts <- rasterToPoints(tau_free_livin_r, spatial=TRUE)
tau_free_livin_df <- as.data.frame(tau_free_livin_pts)
names(tau_free_livin_df)[names(tau_free_livin_df) == 'Free_Livin'] = 'Free_Livin_Raw'
# Loop through to populate the binary column
for (i in 1:nrow(tau_free_livin_df)) {
    if (tau_free_livin_df$Free_Livin_Raw[i] == 0) {
        tau_free_livin_df$Free_Livin_Coral[i] = 0}  
    else { 
        tau_free_livin_df$Free_Livin_Coral[i] = 1
}}
coords <- (tau_free_livin_df[3:4])
tau_free_livin_sp <- SpatialPointsDataFrame(coords, tau_free_livin_df, proj4string = utm_crs)
writeOGR(obj=tau_free_livin_sp, layer="Free_Livin_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=tau_free_livin_sp, layer="Free_Livin_Raw", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)


## Tut
tut_free_livin_r <- rasterize(tut_TOAD[106], tut_r)
tut_free_livin_pts <- rasterToPoints(tut_free_livin_r, spatial=TRUE)
tut_free_livin_df <- as.data.frame(tut_free_livin_pts)
names(tut_free_livin_df)[names(tut_free_livin_df) == 'FREE_LIVIN'] = 'Free_Livin_Raw'
for (i in 1:nrow(tut_free_livin_df)) {
    if (tut_free_livin_df$Free_Livin_Raw[i] == 0) {
        tut_free_livin_df$Free_Livin_Coral[i] = 0}  
    else { 
        tut_free_livin_df$Free_Livin_Coral[i] = 1
}}
coords <- (tut_free_livin_df[3:4])
tut_free_livin_sp <- SpatialPointsDataFrame(coords, tut_free_livin_df, proj4string = utm_crs)
writeOGR(obj=tut_free_livin_sp, layer="Free_Livin_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=tut_free_livin_sp, layer="Free_Livin_Raw", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)


## Ofu-Olo
oo_free_livin_r <- rasterize(oo_TOAD[62], oo_r)
oo_free_livin_pts <- rasterToPoints(oo_free_livin_r, spatial=TRUE)
oo_free_livin_df <- as.data.frame(oo_free_livin_pts)
names(oo_free_livin_df)[names(oo_free_livin_df) == 'Free_Livin'] = 'Free_Livin_Raw'
# Loop through to populate the binary column
for (i in 1:nrow(oo_free_livin_df)) {
    if (oo_free_livin_df$Free_Livin_Raw[i] == 0) {
        oo_free_livin_df$Free_Livin_Coral[i] = 0}  
    else { 
        oo_free_livin_df$Free_Livin_Coral[i] = 1
}}
coords <- (oo_free_livin_df[3:4])
oo_free_livin_sp <- SpatialPointsDataFrame(coords, oo_free_livin_df, proj4string = utm_crs)
writeOGR(obj=oo_free_livin_sp, layer="Free_Livin_Coral", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)
writeOGR(obj=oo_free_livin_sp, layer="Free_Livin_Raw", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD', driver="ESRI Shapefile",
        overwrite_layer=TRUE)

Warning message in writeOGR(obj = tau_free_livin_sp, layer = "Free_Livin_Coral", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = tau_free_livin_sp, layer = "Free_Livin_Raw", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/Tau_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = tut_free_livin_sp, layer = "Free_Livin_Coral", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = tut_free_livin_sp, layer = "Free_Livin_Raw", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/Tut_TOAD", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = oo_free_livin_sp, layer = "Free_Livin_Coral", :
"Field names abbreviated for ESRI Shapefile driver"
Warning message in writeOGR(obj = oo_free_livin_sp, layer = "Free_Livin_Raw", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/OfuOlo_TOAD", :
"Field names abbreviat

#### Absorb predictor data to points
https://stackoverflow.com/questions/42237021/finding-the-nearest-distance-between-two-spatialpointsdataframes-using-gdistance

I'm going to reload the points, convert abundance to raster, and absorb predictor data for each raw (abundance) and converted (binary) raster.

1. SCLER_CORA: Tut
2. MASSIVE: OfuOlo, Tau, Tut
3. PLATE_LIKE: Tut
4. ENCRUSTING: OfuOlo, Tau, Tut
5. BRANCHING: OfuOlo, Tau, Tut
6. COLUMNAR: OfuOlo, Tau, Tut
7. FREE_LIVIN: OfuOlo, Tau, Tut

In [68]:
summary(oo_free_livin_bin)

Object of class SpatialPointsDataFrame
Coordinates:
              min     max
coords.x1  640470  651520
coords.x2 8429775 8436475
Is projected: TRUE 
proj4string :
[+proj=utm +zone=2 +south +ellps=GRS80 +units=m +no_defs]
Number of points: 857
Data attributes:
       ID           Fr_Lv_R           lon              lat         
 Min.   :15216   Min.   : 0.00   Min.   :640470   Min.   :8429775  
 1st Qu.:15934   1st Qu.: 0.00   1st Qu.:642470   1st Qu.:8431275  
 Median :16401   Median : 0.00   Median :646170   Median :8432525  
 Mean   :16565   Mean   : 0.14   Mean   :645853   Mean   :8432910  
 3rd Qu.:17180   3rd Qu.: 0.00   3rd Qu.:648920   3rd Qu.:8434575  
 Max.   :18252   Max.   :40.00   Max.   :651520   Max.   :8436475  
                                                                   
    Fr_Lv_C              bty                hs             rug       
 Min.   :0.000000   Min.   :-185.09   Min.   :1.000   Min.   :1.000  
 1st Qu.:0.000000   1st Qu.: -63.74   1st Qu.:1.043   1

In [36]:
setwd("C:/Users/linds/OneDrive/Documents/samoa_corals_data/")
# Raw (abundance)
# Coral (binary)

## SCLER_CORA: Tut
tut_scler_bin <- readOGR('Tut/Tut_TOAD/Scler_Coral.shp')

## MASSIVE: OfuOlo, Tau, Tut
oo_massive_bin <- readOGR('OfuOlo/OfuOlo_TOAD/Massive_Coral.shp')
tau_massive_bin <- readOGR('Tau/Tau_TOAD/Massive_Coral.shp')
tut_massive_bin <- readOGR('Tut/Tut_TOAD/Massive_Coral.shp')

## PLATE_LIKE: Tut
tut_plate_bin <- readOGR('Tut/Tut_TOAD/Plate_Like_Coral.shp')

## ENCRUSTING: OfuOlo, Tau, Tut
oo_encrusting_bin <- readOGR('OfuOlo/OfuOlo_TOAD/Encrusting_Coral.shp')
tau_encrusting_bin <- readOGR('Tau/Tau_TOAD/Encrusting_Coral.shp')
tut_encrusting_bin <- readOGR('Tut/Tut_TOAD/Encrusting_Coral.shp')

## BRANCHING: OfuOlo, Tau, Tut
oo_branching_bin <- readOGR('OfuOlo/OfuOlo_TOAD/Branching_Coral.shp')
tau_branching_bin <- readOGR('Tau/Tau_TOAD/Branching_Coral.shp')
tut_branching_bin <- readOGR('Tut/Tut_TOAD/Branching_Coral.shp')

## COLUMNAR: OfuOlo, Tau, Tut
oo_columnar_bin <- readOGR('OfuOlo/OfuOlo_TOAD/Columnar_Coral.shp')
tau_columnar_bin <- readOGR('Tau/Tau_TOAD/Columnar_Coral.shp')
tut_columnar_bin <- readOGR('Tut/Tut_TOAD/Columnar_Coral.shp')

## FREE_LIVIN: OfuOlo, Tau, Tut
oo_free_livin_bin <- readOGR('OfuOlo/OfuOlo_TOAD/Free_livin_Coral.shp')
tau_free_livin_bin <- readOGR('Tau/Tau_TOAD/Free_livin_Coral.shp')
tut_free_livin_bin <- readOGR('Tut/Tut_TOAD/Free_livin_Coral.shp')

OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\linds\OneDrive\Documents\samoa_corals_data\Tut\Tut_TOAD\Scler_Coral.shp", layer: "Scler_Coral"
with 1960 features
It has 5 fields
OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\linds\OneDrive\Documents\samoa_corals_data\OfuOlo\OfuOlo_TOAD\Massive_Coral.shp", layer: "Massive_Coral"
with 857 features
It has 5 fields
OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\linds\OneDrive\Documents\samoa_corals_data\Tau\Tau_TOAD\Massive_Coral.shp", layer: "Massive_Coral"
with 346 features
It has 5 fields
OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\linds\OneDrive\Documents\samoa_corals_data\Tut\Tut_TOAD\Massive_Coral.shp", layer: "Massive_Coral"
with 1960 features
It has 5 fields
OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\linds\OneDrive\Documents\samoa_corals_data\Tut\Tut_TOAD\Plate_Like_Coral.shp", layer: "Plate_Like_Coral"
with 1960 features
It has 5 fields
OGR data 

#### Ofu-Olo: MASSIVE, ENCRUSTING, BRANCHING, COLUMNAR, FREE_LIVIN

In [37]:
### MASSIVE

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(oo_bty, oo_massive_bin)
oo_massive_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(oo_hs, oo_massive_bin)
oo_massive_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(oo_rug, oo_massive_bin)
oo_massive_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(oo_slope, oo_massive_bin)
oo_massive_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(oo_asp, oo_massive_bin)
oo_massive_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(oo_massive_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
oo_massive_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(oo_massive_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
oo_massive_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(oo_massive_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(oo_massive_bin[i,], vill_276, byid=TRUE))]}
# add as a column
oo_massive_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(oo_massive_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
oo_massive_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(oo_massive_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(oo_massive_bin[i,], esi, byid=TRUE))]}
oo_massive_bin$esi <- nearestPolys
dist<-gDistance(oo_massive_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
oo_massive_bin$esi_ed <- min_esi_dist

# ucur
n <- length(oo_massive_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(oo_massive_bin[i,], ucur, byid=TRUE))]}
oo_massive_bin$ucur <- nearestPolys

# vcur
n <- length(oo_massive_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(oo_massive_bin[i,], vcur, byid=TRUE))]}
oo_massive_bin$vcur <- nearestPolys

oo_massive_bin$ucur <- as.numeric(oo_massive_bin$ucur)
oo_massive_bin$vcur <- as.numeric(oo_massive_bin$vcur)

# Edit some names
names(oo_massive_bin)[names(oo_massive_bin) == 'x'] = 'lon'
names(oo_massive_bin)[names(oo_massive_bin) == 'y'] = 'lat'

# Convert, write as df
oo_massive_df <- as.data.frame(oo_massive_bin)
write.csv(oo_massive_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/oo_massive_df.csv')

In [38]:
### ENCRUSTING

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(oo_bty, oo_encrusting_bin)
oo_encrusting_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(oo_hs, oo_encrusting_bin)
oo_encrusting_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(oo_rug, oo_encrusting_bin)
oo_encrusting_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(oo_slope, oo_encrusting_bin)
oo_encrusting_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(oo_asp, oo_encrusting_bin)
oo_encrusting_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(oo_encrusting_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
oo_encrusting_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(oo_encrusting_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
oo_encrusting_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(oo_encrusting_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(oo_encrusting_bin[i,], vill_276, byid=TRUE))]}
# add as a column
oo_encrusting_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(oo_encrusting_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
oo_encrusting_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(oo_encrusting_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(oo_encrusting_bin[i,], esi, byid=TRUE))]}
oo_encrusting_bin$esi <- nearestPolys
dist<-gDistance(oo_encrusting_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
oo_encrusting_bin$esi_ed <- min_esi_dist

# ucur
n <- length(oo_encrusting_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(oo_encrusting_bin[i,], ucur, byid=TRUE))]}
oo_encrusting_bin$ucur <- nearestPolys

# vcur
n <- length(oo_encrusting_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(oo_encrusting_bin[i,], vcur, byid=TRUE))]}
oo_encrusting_bin$vcur <- nearestPolys

oo_encrusting_bin$ucur <- as.numeric(oo_encrusting_bin$ucur)
oo_encrusting_bin$vcur <- as.numeric(oo_encrusting_bin$vcur)

# Edit some names
names(oo_encrusting_bin)[names(oo_encrusting_bin) == 'x'] = 'lon'
names(oo_encrusting_bin)[names(oo_encrusting_bin) == 'y'] = 'lat'

# Convert, write as df
oo_encrusting_df <- as.data.frame(oo_encrusting_bin)
write.csv(oo_encrusting_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/oo_encrusting_df.csv')

In [39]:
### BRANCHING

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(oo_bty, oo_branching_bin)
oo_branching_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(oo_hs, oo_branching_bin)
oo_branching_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(oo_rug, oo_branching_bin)
oo_branching_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(oo_slope, oo_branching_bin)
oo_branching_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(oo_asp, oo_branching_bin)
oo_branching_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(oo_branching_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
oo_branching_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(oo_branching_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
oo_branching_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(oo_branching_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(oo_branching_bin[i,], vill_276, byid=TRUE))]}
# add as a column
oo_branching_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(oo_branching_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
oo_branching_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(oo_branching_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(oo_branching_bin[i,], esi, byid=TRUE))]}
oo_branching_bin$esi <- nearestPolys
dist<-gDistance(oo_branching_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
oo_branching_bin$esi_ed <- min_esi_dist

# ucur
n <- length(oo_branching_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(oo_branching_bin[i,], ucur, byid=TRUE))]}
oo_branching_bin$ucur <- nearestPolys

# vcur
n <- length(oo_branching_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(oo_branching_bin[i,], vcur, byid=TRUE))]}
oo_branching_bin$vcur <- nearestPolys

oo_branching_bin$ucur <- as.numeric(oo_branching_bin$ucur)
oo_branching_bin$vcur <- as.numeric(oo_branching_bin$vcur)

# Edit some names
names(oo_branching_bin)[names(oo_branching_bin) == 'x'] = 'lon'
names(oo_branching_bin)[names(oo_branching_bin) == 'y'] = 'lat'

# Convert, write as df
oo_branching_df <- as.data.frame(oo_branching_bin)
write.csv(oo_branching_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/oo_branching_df.csv')

In [40]:
### COLUMNAR

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(oo_bty, oo_columnar_bin)
oo_columnar_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(oo_hs, oo_columnar_bin)
oo_columnar_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(oo_rug, oo_columnar_bin)
oo_columnar_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(oo_slope, oo_columnar_bin)
oo_columnar_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(oo_asp, oo_columnar_bin)
oo_columnar_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(oo_columnar_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
oo_columnar_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(oo_columnar_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
oo_columnar_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(oo_columnar_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(oo_columnar_bin[i,], vill_276, byid=TRUE))]}
# add as a column
oo_columnar_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(oo_columnar_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
oo_columnar_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(oo_columnar_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(oo_columnar_bin[i,], esi, byid=TRUE))]}
oo_columnar_bin$esi <- nearestPolys
dist<-gDistance(oo_columnar_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
oo_columnar_bin$esi_ed <- min_esi_dist

# ucur
n <- length(oo_columnar_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(oo_columnar_bin[i,], ucur, byid=TRUE))]}
oo_columnar_bin$ucur <- nearestPolys

# vcur
n <- length(oo_columnar_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(oo_columnar_bin[i,], vcur, byid=TRUE))]}
oo_columnar_bin$vcur <- nearestPolys

oo_columnar_bin$ucur <- as.numeric(oo_columnar_bin$ucur)
oo_columnar_bin$vcur <- as.numeric(oo_columnar_bin$vcur)

# Edit some names
names(oo_columnar_bin)[names(oo_columnar_bin) == 'x'] = 'lon'
names(oo_columnar_bin)[names(oo_columnar_bin) == 'y'] = 'lat'

# Convert, write as df
oo_columnar_df <- as.data.frame(oo_columnar_bin)
write.csv(oo_columnar_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/oo_columnar_df.csv')

In [41]:
### FREE_LIVIN

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(oo_bty, oo_free_livin_bin)
oo_free_livin_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(oo_hs, oo_free_livin_bin)
oo_free_livin_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(oo_rug, oo_free_livin_bin)
oo_free_livin_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(oo_slope, oo_free_livin_bin)
oo_free_livin_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(oo_asp, oo_free_livin_bin)
oo_free_livin_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(oo_free_livin_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
oo_free_livin_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(oo_free_livin_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
oo_free_livin_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(oo_free_livin_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(oo_free_livin_bin[i,], vill_276, byid=TRUE))]}
# add as a column
oo_free_livin_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(oo_free_livin_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
oo_free_livin_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(oo_free_livin_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(oo_free_livin_bin[i,], esi, byid=TRUE))]}
oo_free_livin_bin$esi <- nearestPolys
dist<-gDistance(oo_free_livin_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
oo_free_livin_bin$esi_ed <- min_esi_dist

# ucur
n <- length(oo_free_livin_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(oo_free_livin_bin[i,], ucur, byid=TRUE))]}
oo_free_livin_bin$ucur <- nearestPolys

# vcur
n <- length(oo_free_livin_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(oo_free_livin_bin[i,], vcur, byid=TRUE))]}
oo_free_livin_bin$vcur <- nearestPolys

oo_free_livin_bin$ucur <- as.numeric(oo_free_livin_bin$ucur)
oo_free_livin_bin$vcur <- as.numeric(oo_free_livin_bin$vcur)

# Edit some names
names(oo_free_livin_bin)[names(oo_free_livin_bin) == 'x'] = 'lon'
names(oo_free_livin_bin)[names(oo_free_livin_bin) == 'y'] = 'lat'

# Convert, write as df
oo_free_livin_df <- as.data.frame(oo_free_livin_bin)
write.csv(oo_free_livin_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/oo_free_livin_df.csv')

#### Tau: MASSIVE, ENCRUSTING, BRANCHING, COLUMNAR, FREE_LIVIN

In [42]:
### MASSIVE

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(tau_bty, tau_massive_bin)
tau_massive_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(tau_hs, tau_massive_bin)
tau_massive_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(tau_rug, tau_massive_bin)
tau_massive_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(tau_slope, tau_massive_bin)
tau_massive_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(tau_asp, tau_massive_bin)
tau_massive_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(tau_massive_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
tau_massive_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(tau_massive_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
tau_massive_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(tau_massive_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(tau_massive_bin[i,], vill_276, byid=TRUE))]}
# add as a column
tau_massive_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(tau_massive_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
tau_massive_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(tau_massive_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(tau_massive_bin[i,], esi, byid=TRUE))]}
tau_massive_bin$esi <- nearestPolys
dist<-gDistance(tau_massive_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
tau_massive_bin$esi_ed <- min_esi_dist

# ucur
n <- length(tau_massive_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(tau_massive_bin[i,], ucur, byid=TRUE))]}
tau_massive_bin$ucur <- nearestPolys

# vcur
n <- length(tau_massive_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(tau_massive_bin[i,], vcur, byid=TRUE))]}
tau_massive_bin$vcur <- nearestPolys

tau_massive_bin$ucur <- as.numeric(tau_massive_bin$ucur)
tau_massive_bin$vcur <- as.numeric(tau_massive_bin$vcur)

# Edit some names
names(tau_massive_bin)[names(tau_massive_bin) == 'x'] = 'lon'
names(tau_massive_bin)[names(tau_massive_bin) == 'y'] = 'lat'

# Convert, write as df
tau_massive_df <- as.data.frame(tau_massive_bin)
write.csv(tau_massive_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/tau_massive_df.csv')

In [43]:
### ENCRUSTING

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(tau_bty, tau_encrusting_bin)
tau_encrusting_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(tau_hs, tau_encrusting_bin)
tau_encrusting_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(tau_rug, tau_encrusting_bin)
tau_encrusting_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(tau_slope, tau_encrusting_bin)
tau_encrusting_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(tau_asp, tau_encrusting_bin)
tau_encrusting_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(tau_encrusting_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
tau_encrusting_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(tau_encrusting_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
tau_encrusting_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(tau_encrusting_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(tau_encrusting_bin[i,], vill_276, byid=TRUE))]}
# add as a column
tau_encrusting_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(tau_encrusting_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
tau_encrusting_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(tau_encrusting_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(tau_encrusting_bin[i,], esi, byid=TRUE))]}
tau_encrusting_bin$esi <- nearestPolys
dist<-gDistance(tau_encrusting_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
tau_encrusting_bin$esi_ed <- min_esi_dist

# ucur
n <- length(tau_encrusting_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(tau_encrusting_bin[i,], ucur, byid=TRUE))]}
tau_encrusting_bin$ucur <- nearestPolys

# vcur
n <- length(tau_encrusting_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(tau_encrusting_bin[i,], vcur, byid=TRUE))]}
tau_encrusting_bin$vcur <- nearestPolys

tau_encrusting_bin$ucur <- as.numeric(tau_encrusting_bin$ucur)
tau_encrusting_bin$vcur <- as.numeric(tau_encrusting_bin$vcur)

# Edit some names
names(tau_encrusting_bin)[names(tau_encrusting_bin) == 'x'] = 'lon'
names(tau_encrusting_bin)[names(tau_encrusting_bin) == 'y'] = 'lat'

# Convert, write as df
tau_encrusting_df <- as.data.frame(tau_encrusting_bin)
write.csv(tau_encrusting_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/tau_encrusting_df.csv')

In [44]:
### BRANCHING

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(tau_bty, tau_branching_bin)
tau_branching_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(tau_hs, tau_branching_bin)
tau_branching_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(tau_rug, tau_branching_bin)
tau_branching_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(tau_slope, tau_branching_bin)
tau_branching_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(tau_asp, tau_branching_bin)
tau_branching_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(tau_branching_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
tau_branching_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(tau_branching_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
tau_branching_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(tau_branching_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(tau_branching_bin[i,], vill_276, byid=TRUE))]}
# add as a column
tau_branching_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(tau_branching_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
tau_branching_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(tau_branching_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(tau_branching_bin[i,], esi, byid=TRUE))]}
tau_branching_bin$esi <- nearestPolys
dist<-gDistance(tau_branching_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
tau_branching_bin$esi_ed <- min_esi_dist

# ucur
n <- length(tau_branching_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(tau_branching_bin[i,], ucur, byid=TRUE))]}
tau_branching_bin$ucur <- nearestPolys

# vcur
n <- length(tau_branching_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(tau_branching_bin[i,], vcur, byid=TRUE))]}
tau_branching_bin$vcur <- nearestPolys

tau_branching_bin$ucur <- as.numeric(tau_branching_bin$ucur)
tau_branching_bin$vcur <- as.numeric(tau_branching_bin$vcur)

# Edit some names
names(tau_branching_bin)[names(tau_branching_bin) == 'x'] = 'lon'
names(tau_branching_bin)[names(tau_branching_bin) == 'y'] = 'lat'

# Convert, write as df
tau_branching_df <- as.data.frame(tau_branching_bin)
write.csv(tau_branching_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/tau_branching_df.csv')

In [45]:
### COLUMNAR

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(tau_bty, tau_columnar_bin)
tau_columnar_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(tau_hs, tau_columnar_bin)
tau_columnar_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(tau_rug, tau_columnar_bin)
tau_columnar_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(tau_slope, tau_columnar_bin)
tau_columnar_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(tau_asp, tau_columnar_bin)
tau_columnar_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(tau_columnar_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
tau_columnar_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(tau_columnar_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
tau_columnar_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(tau_columnar_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(tau_columnar_bin[i,], vill_276, byid=TRUE))]}
# add as a column
tau_columnar_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(tau_columnar_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
tau_columnar_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(tau_columnar_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(tau_columnar_bin[i,], esi, byid=TRUE))]}
tau_columnar_bin$esi <- nearestPolys
dist<-gDistance(tau_columnar_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
tau_columnar_bin$esi_ed <- min_esi_dist

# ucur
n <- length(tau_columnar_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(tau_columnar_bin[i,], ucur, byid=TRUE))]}
tau_columnar_bin$ucur <- nearestPolys

# vcur
n <- length(tau_columnar_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(tau_columnar_bin[i,], vcur, byid=TRUE))]}
tau_columnar_bin$vcur <- nearestPolys

tau_columnar_bin$ucur <- as.numeric(tau_columnar_bin$ucur)
tau_columnar_bin$vcur <- as.numeric(tau_columnar_bin$vcur)

# Edit some names
names(tau_columnar_bin)[names(tau_columnar_bin) == 'x'] = 'lon'
names(tau_columnar_bin)[names(tau_columnar_bin) == 'y'] = 'lat'

# Convert, write as df
tau_columnar_df <- as.data.frame(tau_columnar_bin)
write.csv(tau_columnar_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/tau_columnar_df.csv')

In [46]:
### FREE_LIVIN

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(tau_bty, tau_free_livin_bin)
tau_free_livin_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(tau_hs, tau_free_livin_bin)
tau_free_livin_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(tau_rug, tau_free_livin_bin)
tau_free_livin_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(tau_slope, tau_free_livin_bin)
tau_free_livin_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(tau_asp, tau_free_livin_bin)
tau_free_livin_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(tau_free_livin_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
tau_free_livin_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(tau_free_livin_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
tau_free_livin_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(tau_free_livin_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(tau_free_livin_bin[i,], vill_276, byid=TRUE))]}
# add as a column
tau_free_livin_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(tau_free_livin_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
tau_free_livin_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(tau_free_livin_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(tau_free_livin_bin[i,], esi, byid=TRUE))]}
tau_free_livin_bin$esi <- nearestPolys
dist<-gDistance(tau_free_livin_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
tau_free_livin_bin$esi_ed <- min_esi_dist

# ucur
n <- length(tau_free_livin_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(tau_free_livin_bin[i,], ucur, byid=TRUE))]}
tau_free_livin_bin$ucur <- nearestPolys

# vcur
n <- length(tau_free_livin_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(tau_free_livin_bin[i,], vcur, byid=TRUE))]}
tau_free_livin_bin$vcur <- nearestPolys

tau_free_livin_bin$ucur <- as.numeric(tau_free_livin_bin$ucur)
tau_free_livin_bin$vcur <- as.numeric(tau_free_livin_bin$vcur)

# Edit some names
names(tau_free_livin_bin)[names(tau_free_livin_bin) == 'x'] = 'lon'
names(tau_free_livin_bin)[names(tau_free_livin_bin) == 'y'] = 'lat'

# Convert, write as df
tau_free_livin_df <- as.data.frame(tau_free_livin_bin)
write.csv(tau_free_livin_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/tau_free_livin_df.csv')

#### Tut: MASSIVE, ENCRUSTING, BRANCHING, COLUMNAR, FREE_LIVIN, PLATE_LIKE, SCLER

In [47]:
### MASSIVE

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(tut_bty, tut_massive_bin)
tut_massive_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(tut_hs, tut_massive_bin)
tut_massive_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(tut_rug, tut_massive_bin)
tut_massive_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(tut_slope, tut_massive_bin)
tut_massive_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(tut_asp, tut_massive_bin)
tut_massive_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(tut_massive_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
tut_massive_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(tut_massive_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
tut_massive_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(tut_massive_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(tut_massive_bin[i,], vill_276, byid=TRUE))]}
# add as a column
tut_massive_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(tut_massive_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
tut_massive_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(tut_massive_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(tut_massive_bin[i,], esi, byid=TRUE))]}
tut_massive_bin$esi <- nearestPolys
dist<-gDistance(tut_massive_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
tut_massive_bin$esi_ed <- min_esi_dist

# ucur
n <- length(tut_massive_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(tut_massive_bin[i,], ucur, byid=TRUE))]}
tut_massive_bin$ucur <- nearestPolys

# vcur
n <- length(tut_massive_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(tut_massive_bin[i,], vcur, byid=TRUE))]}
tut_massive_bin$vcur <- nearestPolys

tut_massive_bin$ucur <- as.numeric(tut_massive_bin$ucur)
tut_massive_bin$vcur <- as.numeric(tut_massive_bin$vcur)

# Edit some names
names(tut_massive_bin)[names(tut_massive_bin) == 'x'] = 'lon'
names(tut_massive_bin)[names(tut_massive_bin) == 'y'] = 'lat'

# Convert, write as df
tut_massive_df <- as.data.frame(tut_massive_bin)
write.csv(tut_massive_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/tut_massive_df.csv')

In [48]:
### ENCRUSTING

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(tut_bty, tut_encrusting_bin)
tut_encrusting_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(tut_hs, tut_encrusting_bin)
tut_encrusting_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(tut_rug, tut_encrusting_bin)
tut_encrusting_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(tut_slope, tut_encrusting_bin)
tut_encrusting_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(tut_asp, tut_encrusting_bin)
tut_encrusting_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(tut_encrusting_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
tut_encrusting_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(tut_encrusting_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
tut_encrusting_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(tut_encrusting_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(tut_encrusting_bin[i,], vill_276, byid=TRUE))]}
# add as a column
tut_encrusting_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(tut_encrusting_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
tut_encrusting_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(tut_encrusting_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(tut_encrusting_bin[i,], esi, byid=TRUE))]}
tut_encrusting_bin$esi <- nearestPolys
dist<-gDistance(tut_encrusting_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
tut_encrusting_bin$esi_ed <- min_esi_dist

# ucur
n <- length(tut_encrusting_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(tut_encrusting_bin[i,], ucur, byid=TRUE))]}
tut_encrusting_bin$ucur <- nearestPolys

# vcur
n <- length(tut_encrusting_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(tut_encrusting_bin[i,], vcur, byid=TRUE))]}
tut_encrusting_bin$vcur <- nearestPolys

tut_encrusting_bin$ucur <- as.numeric(tut_encrusting_bin$ucur)
tut_encrusting_bin$vcur <- as.numeric(tut_encrusting_bin$vcur)

# Edit some names
names(tut_encrusting_bin)[names(tut_encrusting_bin) == 'x'] = 'lon'
names(tut_encrusting_bin)[names(tut_encrusting_bin) == 'y'] = 'lat'

# Convert, write as df
tut_encrusting_df <- as.data.frame(tut_encrusting_bin)
write.csv(tut_encrusting_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/tut_encrusting_df.csv')

In [49]:
### BRANCHING

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(tut_bty, tut_branching_bin)
tut_branching_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(tut_hs, tut_branching_bin)
tut_branching_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(tut_rug, tut_branching_bin)
tut_branching_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(tut_slope, tut_branching_bin)
tut_branching_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(tut_asp, tut_branching_bin)
tut_branching_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(tut_branching_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
tut_branching_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(tut_branching_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
tut_branching_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(tut_branching_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(tut_branching_bin[i,], vill_276, byid=TRUE))]}
# add as a column
tut_branching_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(tut_branching_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
tut_branching_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(tut_branching_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(tut_branching_bin[i,], esi, byid=TRUE))]}
tut_branching_bin$esi <- nearestPolys
dist<-gDistance(tut_branching_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
tut_branching_bin$esi_ed <- min_esi_dist

# ucur
n <- length(tut_branching_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(tut_branching_bin[i,], ucur, byid=TRUE))]}
tut_branching_bin$ucur <- nearestPolys

# vcur
n <- length(tut_branching_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(tut_branching_bin[i,], vcur, byid=TRUE))]}
tut_branching_bin$vcur <- nearestPolys

tut_branching_bin$ucur <- as.numeric(tut_branching_bin$ucur)
tut_branching_bin$vcur <- as.numeric(tut_branching_bin$vcur)

# Edit some names
names(tut_branching_bin)[names(tut_branching_bin) == 'x'] = 'lon'
names(tut_branching_bin)[names(tut_branching_bin) == 'y'] = 'lat'

# Convert, write as df
tut_branching_df <- as.data.frame(tut_branching_bin)
write.csv(tut_branching_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/tut_branching_df.csv')

In [50]:
### COLUMNAR

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(tut_bty, tut_columnar_bin)
tut_columnar_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(tut_hs, tut_columnar_bin)
tut_columnar_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(tut_rug, tut_columnar_bin)
tut_columnar_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(tut_slope, tut_columnar_bin)
tut_columnar_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(tut_asp, tut_columnar_bin)
tut_columnar_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(tut_columnar_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
tut_columnar_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(tut_columnar_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
tut_columnar_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(tut_columnar_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(tut_columnar_bin[i,], vill_276, byid=TRUE))]}
# add as a column
tut_columnar_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(tut_columnar_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
tut_columnar_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(tut_columnar_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(tut_columnar_bin[i,], esi, byid=TRUE))]}
tut_columnar_bin$esi <- nearestPolys
dist<-gDistance(tut_columnar_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
tut_columnar_bin$esi_ed <- min_esi_dist

# ucur
n <- length(tut_columnar_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(tut_columnar_bin[i,], ucur, byid=TRUE))]}
tut_columnar_bin$ucur <- nearestPolys

# vcur
n <- length(tut_columnar_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(tut_columnar_bin[i,], vcur, byid=TRUE))]}
tut_columnar_bin$vcur <- nearestPolys

tut_columnar_bin$ucur <- as.numeric(tut_columnar_bin$ucur)
tut_columnar_bin$vcur <- as.numeric(tut_columnar_bin$vcur)

# Edit some names
names(tut_columnar_bin)[names(tut_columnar_bin) == 'x'] = 'lon'
names(tut_columnar_bin)[names(tut_columnar_bin) == 'y'] = 'lat'

# Convert, write as df
tut_columnar_df <- as.data.frame(tut_columnar_bin)
write.csv(tut_columnar_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/tut_columnar_df.csv')

In [51]:
### FREE_LIVIN

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(tut_bty, tut_free_livin_bin)
tut_free_livin_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(tut_hs, tut_free_livin_bin)
tut_free_livin_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(tut_rug, tut_free_livin_bin)
tut_free_livin_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(tut_slope, tut_free_livin_bin)
tut_free_livin_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(tut_asp, tut_free_livin_bin)
tut_free_livin_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(tut_free_livin_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
tut_free_livin_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(tut_free_livin_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
tut_free_livin_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(tut_free_livin_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(tut_free_livin_bin[i,], vill_276, byid=TRUE))]}
# add as a column
tut_free_livin_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(tut_free_livin_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
tut_free_livin_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(tut_free_livin_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(tut_free_livin_bin[i,], esi, byid=TRUE))]}
tut_free_livin_bin$esi <- nearestPolys
dist<-gDistance(tut_free_livin_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
tut_free_livin_bin$esi_ed <- min_esi_dist

# ucur
n <- length(tut_free_livin_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(tut_free_livin_bin[i,], ucur, byid=TRUE))]}
tut_free_livin_bin$ucur <- nearestPolys

# vcur
n <- length(tut_free_livin_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(tut_free_livin_bin[i,], vcur, byid=TRUE))]}
tut_free_livin_bin$vcur <- nearestPolys

tut_free_livin_bin$ucur <- as.numeric(tut_free_livin_bin$ucur)
tut_free_livin_bin$vcur <- as.numeric(tut_free_livin_bin$vcur)

# Edit some names
names(tut_free_livin_bin)[names(tut_free_livin_bin) == 'x'] = 'lon'
names(tut_free_livin_bin)[names(tut_free_livin_bin) == 'y'] = 'lat'

# Convert, write as df
tut_free_livin_df <- as.data.frame(tut_free_livin_bin)
write.csv(tut_free_livin_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/tut_free_livin_df.csv')

In [52]:
### PLATE

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(tut_bty, tut_plate_bin)
tut_plate_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(tut_hs, tut_plate_bin)
tut_plate_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(tut_rug, tut_plate_bin)
tut_plate_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(tut_slope, tut_plate_bin)
tut_plate_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(tut_asp, tut_plate_bin)
tut_plate_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(tut_plate_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
tut_plate_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(tut_plate_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
tut_plate_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(tut_plate_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(tut_plate_bin[i,], vill_276, byid=TRUE))]}
# add as a column
tut_plate_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(tut_plate_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
tut_plate_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(tut_plate_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(tut_plate_bin[i,], esi, byid=TRUE))]}
tut_plate_bin$esi <- nearestPolys
dist<-gDistance(tut_plate_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
tut_plate_bin$esi_ed <- min_esi_dist

# ucur
n <- length(tut_plate_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(tut_plate_bin[i,], ucur, byid=TRUE))]}
tut_plate_bin$ucur <- nearestPolys

# vcur
n <- length(tut_plate_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(tut_plate_bin[i,], vcur, byid=TRUE))]}
tut_plate_bin$vcur <- nearestPolys

tut_plate_bin$ucur <- as.numeric(tut_plate_bin$ucur)
tut_plate_bin$vcur <- as.numeric(tut_plate_bin$vcur)

# Edit some names
names(tut_plate_bin)[names(tut_plate_bin) == 'x'] = 'lon'
names(tut_plate_bin)[names(tut_plate_bin) == 'y'] = 'lat'

# Convert, write as df
tut_plate_df <- as.data.frame(tut_plate_bin)
write.csv(tut_plate_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/tut_plate_df.csv')

In [53]:
### SCLER

# Extract raster data to pts
# depth
bty_tmp <- raster::extract(tut_bty, tut_scler_bin)
tut_scler_bin$bty <- bty_tmp
# backscatter
hs_tmp <- raster::extract(tut_hs, tut_scler_bin)
tut_scler_bin$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(tut_rug, tut_scler_bin)
tut_scler_bin$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(tut_slope, tut_scler_bin)
tut_scler_bin$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(tut_asp, tut_scler_bin)
tut_scler_bin$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(tut_scler_bin, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
tut_scler_bin$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(tut_scler_bin, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
tut_scler_bin$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(tut_scler_bin)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(tut_scler_bin[i,], vill_276, byid=TRUE))]}
# add as a column
tut_scler_bin$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(tut_scler_bin, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
tut_scler_bin$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(tut_scler_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(tut_scler_bin[i,], esi, byid=TRUE))]}
tut_scler_bin$esi <- nearestPolys
dist<-gDistance(tut_scler_bin, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
tut_scler_bin$esi_ed <- min_esi_dist

# ucur
n <- length(tut_scler_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(tut_scler_bin[i,], ucur, byid=TRUE))]}
tut_scler_bin$ucur <- nearestPolys

# vcur
n <- length(tut_scler_bin)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(tut_scler_bin[i,], vcur, byid=TRUE))]}
tut_scler_bin$vcur <- nearestPolys

tut_scler_bin$ucur <- as.numeric(tut_scler_bin$ucur)
tut_scler_bin$vcur <- as.numeric(tut_scler_bin$vcur)

# Edit some names
names(tut_scler_bin)[names(tut_scler_bin) == 'x'] = 'lon'
names(tut_scler_bin)[names(tut_scler_bin) == 'y'] = 'lat'

# Convert, write as df
tut_scler_df <- as.data.frame(tut_scler_bin)
write.csv(tut_scler_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/tut_scler_df.csv')

### Regional predictive dataframes

For this stage, I need to...
- [done] Convert the bty raster to pts for each region: OfuOlu, Tau, Tut, Rose, Swains
- [done] Extract all predictor data to pts for each region
- [done] Convert pts to dfs
- [done] Save all dfs, pt shps in their respective home directories

In [54]:
setwd("C:/Users/linds/OneDrive/Documents/samoa_corals_data/")

library(data.table)
library(geojsonio)
library(GISTools)
library(ncdf4)
library(raster)
library(rgdal)
library(rgeos)
library(sf)
library(spatial.tools)
library(tidyverse)

# Ofu-Olo
oo_bty <-raster('OfuOlo/OfuOlo_bty/oo_bty.grd')
oo_hs <- raster('OfuOlo/OfuOlo_hs/oo_hs.grd')
oo_rug <- raster('OfuOlo/OfuOlo_rugosity/oo_rug.grd')
oo_slope <- raster('OfuOlo/OfuOlo_slope/oo_slope.grd')
oo_asp <- raster('OfuOlo/OfuOlo_asp/oo_aspect.grd')
oo_TOAD <- readOGR('OfuOlo/OfuOlo_TOAD/oo_TOAD.shp') 

# Tau
tau_bty <-raster('Tau/Tau_bty/tau_bty.grd')
tau_hs <- raster('Tau/Tau_hs/tau_hs.grd')
tau_rug <- raster('Tau/Tau_rugosity/tau_rug.grd')
tau_slope <- raster('Tau/Tau_slope/tau_slope.grd')
tau_asp <- raster('Tau/Tau_asp/tau_aspect.grd')
tau_TOAD <- readOGR('Tau/Tau_TOAD/tau_TOAD.shp') 

# Tut
tut_bty <-raster('Tut/tut_bty/tut_bty.grd')
tut_hs <- raster('Tut/tut_hs/tut_hs.grd')
tut_rug <- raster('Tut/tut_rugosity/tut_rug.grd')
tut_slope <- raster('Tut/tut_slope/tut_slope.grd')
tut_asp <- raster('Tut/tut_asp/tut_aspect.grd')
tut_TOAD <- readOGR('Tut/tut_TOAD/tut_TOAD.shp') 

# Rose 
rose_bty <-raster('Rose/rose_bty/rose_bty.grd')
rose_hs <- raster('Rose/rose_hs/rose_hs.grd')
rose_rug <- raster('Rose/rose_rugosity/rose_rug.grd')
rose_slope <- raster('Rose/rose_slope/rose_slope.grd')
rose_asp <- raster('Rose/rose_asp/rose_aspect.grd')

# Swains
swains_bty <-raster('Swains/swains_bty/swains_bty.grd')
swains_hs <- raster('Swains/swains_hs/swains_hs.grd')
swains_rug <- raster('Swains/swains_rugosity/swains_rug.grd')
swains_slope <- raster('Swains/swains_slope/swains_slope.grd')
swains_asp <- raster('Swains/swains_asp/swains_aspect.grd')

# All
ucur <- readOGR('AS_all/Currents/ucur.shp')
vcur <- readOGR('AS_all/Currents/vcur.shp')
esi <- readOGR('AS_all/Env_sens_index/esi.shp')
shore <- readOGR('AS_all/Shorelines/shore.shp')
villages <- readOGR('AS_all/Villages/villages.shp')
wetlands <- readOGR('AS_all/Wetlands/wetlands.shp')

OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\linds\OneDrive\Documents\samoa_corals_data\OfuOlo\OfuOlo_TOAD\oo_TOAD.shp", layer: "oo_TOAD"
with 18252 features
It has 97 fields
OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\linds\OneDrive\Documents\samoa_corals_data\Tau\Tau_TOAD\tau_TOAD.shp", layer: "tau_TOAD"
with 5104 features
It has 97 fields
OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\linds\OneDrive\Documents\samoa_corals_data\Tut\Tut_TOAD\tut_TOAD.shp", layer: "tut_TOAD"
with 33612 features
It has 107 fields
OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\linds\OneDrive\Documents\samoa_corals_data\AS_all\Currents\ucur.shp", layer: "ucur"
with 61077 features
It has 3 fields
OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\linds\OneDrive\Documents\samoa_corals_data\AS_all\Currents\vcur.shp", layer: "vcur"
with 61077 features
It has 3 fields
OGR data source with driver: ESRI Shapefile 
Source: "C:\Users\

#### Ofu-Olo: regional df

In [55]:
# Convert bty raster to pts
oo_bty_pts <- rasterToPoints(oo_bty, spatial=TRUE)

# Extract raster data to pts
# backscatter
hs_tmp <- raster::extract(oo_hs, oo_bty_pts)
oo_bty_pts$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(oo_rug, oo_bty_pts)
oo_bty_pts$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(oo_slope, oo_bty_pts)
oo_bty_pts$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(oo_asp, oo_bty_pts)
oo_bty_pts$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(oo_bty_pts, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
oo_bty_pts$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(oo_bty_pts, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
oo_bty_pts$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(oo_bty_pts)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(oo_bty_pts[i,], vill_276, byid=TRUE))]}
# add as a column
oo_bty_pts$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(oo_bty_pts, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
oo_bty_pts$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(oo_bty_pts)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(oo_bty_pts[i,], esi, byid=TRUE))]}
oo_bty_pts$esi <- nearestPolys
dist<-gDistance(oo_bty_pts, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
oo_bty_pts$esi_ed <- min_esi_dist

# ucur
n <- length(oo_bty_pts)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(oo_bty_pts[i,], ucur, byid=TRUE))]}
oo_bty_pts$ucur <- nearestPolys

# vcur
n <- length(oo_bty_pts)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(oo_bty_pts[i,], vcur, byid=TRUE))]}
oo_bty_pts$vcur <- nearestPolys

oo_bty_pts$ucur <- as.numeric(oo_bty_pts$ucur)
oo_bty_pts$vcur <- as.numeric(oo_bty_pts$vcur)

# Edit some names
names(oo_bty_pts)[names(oo_bty_pts) == 'x'] = 'lon'
names(oo_bty_pts)[names(oo_bty_pts) == 'y'] = 'lat'
names(oo_bty_pts)[names(oo_bty_pts) == 'oo_dbmb_mos4'] = 'depth'
oo_bty_pts$dummy <- 'NA'

# Write as a shapefile
writeOGR(obj=oo_bty_pts, layer="dummy", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo', driver="ESRI Shapefile",
         overwrite_layer=TRUE)

# Convert, write as df
oo_bty_pts_df <- as.data.frame(oo_bty_pts)
write.csv(oo_bty_pts_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo/oo_bty_pts.csv')

Warning message in writeOGR(obj = oo_bty_pts, layer = "dummy", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo", :
"Field names abbreviated for ESRI Shapefile driver"


#### Tau: regional df

In [56]:
# Convert bty raster to pts
tau_bty_pts <- rasterToPoints(tau_bty, spatial=TRUE)

# Extract raster data to pts
# backscatter
hs_tmp <- raster::extract(tau_hs, tau_bty_pts)
tau_bty_pts$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(tau_rug, tau_bty_pts)
tau_bty_pts$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(tau_slope, tau_bty_pts)
tau_bty_pts$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(tau_asp, tau_bty_pts)
tau_bty_pts$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(tau_bty_pts, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
tau_bty_pts$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(tau_bty_pts, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
tau_bty_pts$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(tau_bty_pts)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(tau_bty_pts[i,], vill_276, byid=TRUE))]}
# add as a column
tau_bty_pts$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(tau_bty_pts, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
tau_bty_pts$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(tau_bty_pts)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(tau_bty_pts[i,], esi, byid=TRUE))]}
tau_bty_pts$esi <- nearestPolys
dist<-gDistance(tau_bty_pts, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
tau_bty_pts$esi_ed <- min_esi_dist

# ucur
n <- length(tau_bty_pts)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(tau_bty_pts[i,], ucur, byid=TRUE))]}
tau_bty_pts$ucur <- nearestPolys

# vcur
n <- length(tau_bty_pts)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(tau_bty_pts[i,], vcur, byid=TRUE))]}
tau_bty_pts$vcur <- nearestPolys

tau_bty_pts$ucur <- as.numeric(tau_bty_pts$ucur)
tau_bty_pts$vcur <- as.numeric(tau_bty_pts$vcur)

# Edit some names
names(tau_bty_pts)[names(tau_bty_pts) == 'x'] = 'lon'
names(tau_bty_pts)[names(tau_bty_pts) == 'y'] = 'lat'
names(tau_bty_pts)[names(tau_bty_pts) == 'tau_dbmb_mos4'] = 'depth'
tau_bty_pts$dummy <- 'NA'

# Write as a shapefile
writeOGR(obj=tau_bty_pts, layer="dummy", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau', driver="ESRI Shapefile",
         overwrite_layer=TRUE)

# Convert, write as df
tau_bty_pts_df <- as.data.frame(tau_bty_pts)
write.csv(tau_bty_pts_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau/tau_bty_pts.csv')

Warning message in writeOGR(obj = tau_bty_pts, layer = "dummy", dsn = "C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau", :
"Field names abbreviated for ESRI Shapefile driver"


#### Tutuila: regional df

In [57]:
# Convert bty raster to pts
tut_bty_pts <- rasterToPoints(tut_bty, spatial=TRUE)

# Extract raster data to pts
# backscatter
hs_tmp <- raster::extract(tut_hs, tut_bty_pts)
tut_bty_pts$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(tut_rug, tut_bty_pts)
tut_bty_pts$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(tut_slope, tut_bty_pts)
tut_bty_pts$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(tut_asp, tut_bty_pts)
tut_bty_pts$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(tut_bty_pts, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
tut_bty_pts$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(tut_bty_pts, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
tut_bty_pts$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(tut_bty_pts)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(tut_bty_pts[i,], vill_276, byid=TRUE))]}
# add as a column
tut_bty_pts$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(tut_bty_pts, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
tut_bty_pts$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(tut_bty_pts)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(tut_bty_pts[i,], esi, byid=TRUE))]}
tut_bty_pts$esi <- nearestPolys
dist<-gDistance(tut_bty_pts, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
tut_bty_pts$esi_ed <- min_esi_dist

# ucur
n <- length(tut_bty_pts)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(tut_bty_pts[i,], ucur, byid=TRUE))]}
tut_bty_pts$ucur <- nearestPolys

# vcur
n <- length(tut_bty_pts)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(tut_bty_pts[i,], vcur, byid=TRUE))]}
tut_bty_pts$vcur <- nearestPolys

tut_bty_pts$ucur <- as.numeric(tut_bty_pts$ucur)
tut_bty_pts$vcur <- as.numeric(tut_bty_pts$vcur)

# Edit some names
names(tut_bty_pts)[names(tut_bty_pts) == 'x'] = 'lon'
names(tut_bty_pts)[names(tut_bty_pts) == 'y'] = 'lat'
names(tut_bty_pts)[names(tut_bty_pts) == 'tut_dbmb_mos4'] = 'depth'
tut_bty_pts$dummy <- 'NA'

# Write as a shapefile
writeOGR(obj=tut_bty_pts, layer="dummy", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut', driver="ESRI Shapefile",
         overwrite_layer=TRUE)

# Convert, write as df
tut_bty_pts_df <- as.data.frame(tut_bty_pts)
write.csv(tut_bty_pts_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut/tut_bty_pts.csv')

ERROR: Error: cannot allocate vector of size 2.1 Gb


#### Rose: regional df

In [ ]:
# Convert bty raster to pts
rose_bty_pts <- rasterToPoints(rose_bty, spatial=TRUE)

# Extract raster data to pts
# backscatter
hs_tmp <- raster::extract(rose_hs, rose_bty_pts)
rose_bty_pts$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(rose_rug, rose_bty_pts)
rose_bty_pts$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(rose_slope, rose_bty_pts)
rose_bty_pts$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(rose_asp, rose_bty_pts)
rose_bty_pts$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(rose_bty_pts, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
rose_bty_pts$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(rose_bty_pts, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
rose_bty_pts$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(rose_bty_pts)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(rose_bty_pts[i,], vill_276, byid=TRUE))]}
# add as a column
rose_bty_pts$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(rose_bty_pts, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
rose_bty_pts$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(rose_bty_pts)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(rose_bty_pts[i,], esi, byid=TRUE))]}
rose_bty_pts$esi <- nearestPolys
dist<-gDistance(rose_bty_pts, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
rose_bty_pts$esi_ed <- min_esi_dist

# ucur
n <- length(rose_bty_pts)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(rose_bty_pts[i,], ucur, byid=TRUE))]}
rose_bty_pts$ucur <- nearestPolys

# vcur
n <- length(rose_bty_pts)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(rose_bty_pts[i,], vcur, byid=TRUE))]}
rose_bty_pts$vcur <- nearestPolys

rose_bty_pts$ucur <- as.numeric(rose_bty_pts$ucur)
rose_bty_pts$vcur <- as.numeric(rose_bty_pts$vcur)

# Edit some names
names(rose_bty_pts)[names(rose_bty_pts) == 'x'] = 'lon'
names(rose_bty_pts)[names(rose_bty_pts) == 'y'] = 'lat'
names(rose_bty_pts)[names(rose_bty_pts) == 'rose_dbmb_mos4'] = 'depth'
rose_bty_pts$dummy <- 'NA'

# Write as a shapefile
writeOGR(obj=rose_bty_pts, layer="dummy", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Rose', driver="ESRI Shapefile",
         overwrite_layer=TRUE)

# Convert, write as df
rose_bty_pts_df <- as.data.frame(rose_bty_pts)
write.csv(rose_bty_pts_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/Rose/rose_bty_pts.csv')

#### Swains: regional df

In [ ]:
# Convert bty raster to pts
swains_bty_pts <- rasterToPoints(swains_bty, spatial=TRUE)

# Extract raster data to pts
# backscatter
hs_tmp <- raster::extract(swains_hs, swains_bty_pts)
swains_bty_pts$hs <- hs_tmp
# rugosity
rug_tmp <- raster::extract(swains_rug, swains_bty_pts)
swains_bty_pts$rug <- rug_tmp
# slope
slope_tmp <- raster::extract(swains_slope, swains_bty_pts)
swains_bty_pts$slope <- slope_tmp
# aspect
asp_tmp <- raster::extract(swains_asp, swains_bty_pts)
swains_bty_pts$asp <- asp_tmp

# Calculate distance to shore, wetlands
# shore
dist<-gDistance(swains_bty_pts, shore, byid=T)
min_shore_dist<-apply(dist, 2, min)
swains_bty_pts$shore_ed<-min_shore_dist

# wetlands
dist<-gDistance(swains_bty_pts, wetlands, byid=T)
min_wl_dist<-apply(dist, 2, min)
swains_bty_pts$wl_ed<-min_wl_dist

# Extract values from nearest shps
# villages: only considering villages > 3rd quantile in acres (276)
vill_276 <- subset(villages, villages$ACRES > 276)
# set up container for results
n <- length(swains_bty_pts)
nearestPolys <- character(n)
# for each point, find name of nearest polygon 
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vill_276$ACRES[which.min(gDistance(swains_bty_pts[i,], vill_276, byid=TRUE))]}
# add as a column
swains_bty_pts$vill_third_q <- nearestPolys
# calculate distance to polygon for this variable
dist<-gDistance(swains_bty_pts, vill_276, byid=T)
min_vill_dist<-apply(dist, 2, min)
swains_bty_pts$vill_ed <- min_vill_dist

# esi: change ESI type from fct to chr
esi$ESI <- as.character(esi$ESI)
n <- length(swains_bty_pts)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- esi$ESI[which.min(gDistance(swains_bty_pts[i,], esi, byid=TRUE))]}
swains_bty_pts$esi <- nearestPolys
dist<-gDistance(swains_bty_pts, esi, byid=T)
min_esi_dist<-apply(dist, 2, min)
swains_bty_pts$esi_ed <- min_esi_dist

# ucur
n <- length(swains_bty_pts)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- ucur$umean[which.min(gDistance(swains_bty_pts[i,], ucur, byid=TRUE))]}
swains_bty_pts$ucur <- nearestPolys

# vcur
n <- length(swains_bty_pts)
nearestPolys <- character(n)
for (i in seq_along(nearestPolys)) {
    nearestPolys[i] <- vcur$vmean[which.min(gDistance(swains_bty_pts[i,], vcur, byid=TRUE))]}
swains_bty_pts$vcur <- nearestPolys

swains_bty_pts$ucur <- as.numeric(swains_bty_pts$ucur)
swains_bty_pts$vcur <- as.numeric(swains_bty_pts$vcur)

# Edit some names
names(swains_bty_pts)[names(swains_bty_pts) == 'x'] = 'lon'
names(swains_bty_pts)[names(swains_bty_pts) == 'y'] = 'lat'
names(swains_bty_pts)[names(swains_bty_pts) == 'swains_dbmb_mos4'] = 'depth'
swains_bty_pts$dummy <- 'NA'

# Write as a shapefile
writeOGR(obj=swains_bty_pts, layer="dummy", 
         dsn='C:/Users/linds/OneDrive/Documents/samoa_corals_data/Swains', driver="ESRI Shapefile",
         overwrite_layer=TRUE)

# Convert, write as df
swains_bty_pts_df <- as.data.frame(swains_bty_pts)
write.csv(swains_bty_pts_df, 'C:/Users/linds/OneDrive/Documents/samoa_corals_data/Swains/swains_bty_pts.csv')

In [ ]:
### Plot rasters

#histogram(tmin1.c)
#pairs(tmin.all.c)

### Autocorrelation
# Moran(tmin1.c)  # global Moran's I
#tmin1.Moran <- MoranLocal(tmin1.c)
#plot(tmin1.Moran)

#### Notes etc.

- Literature review focusing on priority, then secondary, locations; environmental characteristics of the mesophotic in these areas; mesophotic ecology
- [GeoViews walkthrough for visualization](https://towardsdatascience.com/the-5-classification-evaluation-metrics-you-must-know-aa97784ff226)
- Models: standard GLM classifier, random forest classifier, possibly XGBoost or neural network
- [Metrics for evaluation](https://towardsdatascience.com/the-5-classification-evaluation-metrics-you-must-know-aa97784ff226)
- [Make nice maps](https://www.benjaminbell.co.uk/2019/08/creating-simple-location-maps-in-r.html)